In [1]:
%matplotlib inline
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.signal import savgol_filter
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc
from scipy.stats import linregress


def get_data_from_mat_for_calcu(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:]
    return var.T

def get_data_from_nc(file, variable):
    file_obj = nc.Dataset(file)
    data = file_obj.variables[variable]
    var_data = np.array(data)
    var_data = var_data[:, 29:, :]
    var_data[var_data > 1000000] = np.nan
    var_data = np.squeeze(var_data)
    return var_data

def get_data_from_mat(file, variable):
    var_dict = scio.loadmat(file)
    var = var_dict[variable]
    var = var[:, 29:] 
    return var

ar6_region = get_data_from_mat_for_calcu('ar6_region.mat','ar6_region')
AREA = get_data_from_mat('input_data/AREA.mat', 'AREA') # Area for CLM_data
irr_diff = get_data_from_mat_for_calcu('irr_diff_out.mat', 'irr_diff_out')

In [2]:
data_surface = Data_from_nc('surfdata_irrigation_method.nc')   #load the data
data_irrigation_method = data_surface.load_variable('irrigation_method')
data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[29:,0]
data_lat = data_lat[:]


def get_data_CESM2(variable):
    
    IWW_IRR_1901_2014_CESM2_IRR01 = get_data_from_nc('CESM2' + '_IRR01_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_NOI01 = get_data_from_nc('CESM2' + '_NOI01_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_IRR02 = get_data_from_nc('CESM2' + '_IRR02_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_NOI02 = get_data_from_nc('CESM2' + '_NOI02_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_IRR03 = get_data_from_nc('CESM2' + '_IRR03_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_NOI03 = get_data_from_nc('CESM2' + '_NOI03_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2 = (IWW_IRR_1901_2014_CESM2_IRR01 + IWW_IRR_1901_2014_CESM2_IRR02 + IWW_IRR_1901_2014_CESM2_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_CESM2 = (IWW_NOI_1901_2014_CESM2_NOI01 + IWW_NOI_1901_2014_CESM2_NOI02 + IWW_NOI_1901_2014_CESM2_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_CESM2[:-1,:,:], IWW_NOI_1901_2014_CESM2[:-1,:,:]

def get_data_E3SM(variable3):
    
    IWW_IRR_1901_2014_E3SM_IRR01 = get_data_from_nc('E3SM_IRR01_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)
    IWW_NOI_1901_2014_E3SM_NOI01 = get_data_from_nc('E3SM_NOI01_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)

    IWW_IRR_1901_2014_E3SM_IRR02 = get_data_from_nc('E3SM_IRR02_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)
    IWW_NOI_1901_2014_E3SM_NOI02 = get_data_from_nc('E3SM_NOI02_1901_2014_'+ variable3 + '_yearmean_0.9x1.25', variable3)

    IWW_IRR_1901_2014_E3SM = (IWW_IRR_1901_2014_E3SM_IRR01 + IWW_IRR_1901_2014_E3SM_IRR02) / 2 * 365 * 86400
    IWW_NOI_1901_2014_E3SM = (IWW_NOI_1901_2014_E3SM_NOI01 + IWW_NOI_1901_2014_E3SM_NOI02) / 2 * 365 * 86400

    return IWW_IRR_1901_2014_E3SM[6:-1,:,:], IWW_NOI_1901_2014_E3SM[6:-1,:,:]

def get_data_CESM2_gw(variable):
    
    IWW_IRR_1901_2014_CESM2_gw_IRR01 = get_data_from_nc('CESM2_gw' + '_IRR01_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_gw_NOI01 = get_data_from_nc('CESM2_gw' + '_NOI01_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_gw_IRR02 = get_data_from_nc('CESM2_gw' + '_IRR02_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_gw_NOI02 = get_data_from_nc('CESM2_gw' + '_NOI02_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_gw_IRR03 = get_data_from_nc('CESM2_gw' + '_IRR03_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_CESM2_gw_NOI03 = get_data_from_nc('CESM2_gw' + '_NOI03_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_CESM2_gw = (IWW_IRR_1901_2014_CESM2_gw_IRR01 + IWW_IRR_1901_2014_CESM2_gw_IRR02 + IWW_IRR_1901_2014_CESM2_gw_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_CESM2_gw = (IWW_NOI_1901_2014_CESM2_gw_NOI01 + IWW_NOI_1901_2014_CESM2_gw_NOI02 + IWW_NOI_1901_2014_CESM2_gw_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_CESM2_gw[:-1,:,:], IWW_NOI_1901_2014_CESM2_gw[:-1,:,:]
    
def get_data_NorESM(variable):
    IWW_IRR_1901_2014_NorESM_IRR01 = get_data_from_nc('NorESM' + '_IRR01_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_NorESM_NOI01 = get_data_from_nc('NorESM' + '_NOI01_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_NorESM_IRR02 = get_data_from_nc('NorESM' + '_IRR02_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_NorESM_NOI02 = get_data_from_nc('NorESM' + '_NOI02_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_NorESM_IRR03 = get_data_from_nc('NorESM' + '_IRR03_1901_2014_'+ variable + '_yearmean', variable)
    IWW_NOI_1901_2014_NorESM_NOI03 = get_data_from_nc('NorESM' + '_NOI03_1901_2014_'+ variable + '_yearmean', variable)

    IWW_IRR_1901_2014_NorESM_IRR01_temp = np.zeros([116,163,288])
    IWW_IRR_1901_2014_NorESM_IRR01_temp[1:, :, :] = IWW_IRR_1901_2014_NorESM_IRR01
    IWW_IRR_1901_2014_NorESM_IRR01_temp[0, :, :] = (IWW_IRR_1901_2014_NorESM_IRR02[0, :, :] + IWW_IRR_1901_2014_NorESM_IRR03[0, :, :])/2
    IWW_IRR_1901_2014_NorESM_IRR01 = IWW_IRR_1901_2014_NorESM_IRR01_temp
    
    IWW_IRR_1901_2014_NorESM = (IWW_IRR_1901_2014_NorESM_IRR01 + IWW_IRR_1901_2014_NorESM_IRR02 + IWW_IRR_1901_2014_NorESM_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_NorESM = (IWW_NOI_1901_2014_NorESM_NOI01 + IWW_NOI_1901_2014_NorESM_NOI02 + IWW_NOI_1901_2014_NorESM_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_NorESM[1:-1,:,:], IWW_NOI_1901_2014_NorESM[1:-1,:,:]
    
    
    
def get_data_IPSL(variable2):
    
    IWW_IRR_1901_2014_IPSL = get_data_from_nc('IRR01_'+ variable2 + '_1901_2014_Month.nc_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_IPSL = get_data_from_nc('NOI01_'+ variable2 + '_1901_2014_Month.nc_yearmean_0.9x1.25', variable2)


    IWW_IRR_1901_2014_IPSL = (IWW_IRR_1901_2014_IPSL) * 365 * 86400
    IWW_NOI_1901_2014_IPSL = (IWW_NOI_1901_2014_IPSL) * 365 * 86400

    return IWW_IRR_1901_2014_IPSL, IWW_NOI_1901_2014_IPSL

def get_data_CNRM(variable2):
    
    IWW_IRR_1901_2014_CNRM_IRR01 = get_data_from_nc(variable2 + '_IRR01.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI01 = get_data_from_nc(variable2 + '_NOI01.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR02 = get_data_from_nc(variable2 + '_IRR02.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI02 = get_data_from_nc(variable2 + '_NOI02.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR03 = get_data_from_nc(variable2 + '_IRR03.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI03 = get_data_from_nc(variable2 + '_NOI03.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR04 = get_data_from_nc(variable2 + '_IRR04.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI04 = get_data_from_nc(variable2 + '_NOI04.nc_yearmean_yearmean_0.9x1.25', variable2)

    IWW_IRR_1901_2014_CNRM_IRR05 = get_data_from_nc(variable2 + '_IRR05.nc_yearmean_yearmean_0.9x1.25', variable2)
    IWW_NOI_1901_2014_CNRM_NOI05 = get_data_from_nc(variable2 + '_NOI05.nc_yearmean_yearmean_0.9x1.25', variable2)

 
    
    IWW_IRR_1901_2014_CNRM = (IWW_IRR_1901_2014_CNRM_IRR01+IWW_IRR_1901_2014_CNRM_IRR02+IWW_IRR_1901_2014_CNRM_IRR03+IWW_IRR_1901_2014_CNRM_IRR04+IWW_IRR_1901_2014_CNRM_IRR05) / 5 * 365 * 86400
    IWW_NOI_1901_2014_CNRM = (IWW_NOI_1901_2014_CNRM_NOI01+IWW_NOI_1901_2014_CNRM_NOI02+IWW_NOI_1901_2014_CNRM_NOI03+IWW_NOI_1901_2014_CNRM_NOI04+IWW_NOI_1901_2014_CNRM_NOI05) / 5 * 365 * 86400

#     IWW_IRR_1901_2014_CNRM = (IWW_IRR_1901_2014_CNRM_IRR02+IWW_IRR_1901_2014_CNRM_IRR03+IWW_IRR_1901_2014_CNRM_IRR04+IWW_IRR_1901_2014_CNRM_IRR05) / 4 * 365 * 86400
#     IWW_NOI_1901_2014_CNRM = (IWW_NOI_1901_2014_CNRM_NOI02+IWW_NOI_1901_2014_CNRM_NOI03+IWW_NOI_1901_2014_CNRM_NOI04+IWW_NOI_1901_2014_CNRM_NOI05) / 4 * 365 * 86400

    return IWW_IRR_1901_2014_CNRM, IWW_NOI_1901_2014_CNRM
    
    
def get_data_MIROC(variable):
    
    IWW_IRR_1901_2014_MIROC_IRR01 = get_data_from_nc(variable + '_mon_MIROC_IRR01_1901-2014.nc_0.9x1.25_yearmean', variable)
    IWW_NOI_1901_2014_MIROC_NOI01 = get_data_from_nc(variable + '_mon_MIROC_NOI01_1901-2014.nc_0.9x1.25_yearmean', variable)

    IWW_IRR_1901_2014_MIROC_IRR02 = get_data_from_nc(variable + '_mon_MIROC_IRR02_1901-2014.nc_0.9x1.25_yearmean', variable)
    IWW_NOI_1901_2014_MIROC_NOI02 = get_data_from_nc(variable + '_mon_MIROC_NOI02_1901-2014.nc_0.9x1.25_yearmean', variable)

    IWW_IRR_1901_2014_MIROC_IRR03 = get_data_from_nc(variable + '_mon_MIROC_IRR03_1901-2014.nc_0.9x1.25_yearmean', variable)
    IWW_NOI_1901_2014_MIROC_NOI03 = get_data_from_nc(variable + '_mon_MIROC_NOI03_1901-2014.nc_0.9x1.25_yearmean', variable)

    IWW_IRR_1901_2014_MIROC = (IWW_IRR_1901_2014_MIROC_IRR01 + IWW_IRR_1901_2014_MIROC_IRR02 + IWW_IRR_1901_2014_MIROC_IRR03) / 3 * 365 * 86400
    IWW_NOI_1901_2014_MIROC = (IWW_NOI_1901_2014_MIROC_NOI01 + IWW_NOI_1901_2014_MIROC_NOI02 + IWW_NOI_1901_2014_MIROC_NOI03) / 3 * 365 * 86400
    
    return IWW_IRR_1901_2014_MIROC, IWW_NOI_1901_2014_MIROC

#     All_forcings_all = (All_forcings_CESM2 + All_forcings_CESM2_gw + All_forcings_NorESM + All_forcings_CNRM + All_forcings_IPSL) / 6
#     Oth_forcings_all = (Oth_forcings_CESM2 + Oth_forcings_CESM2_gw + Oth_forcings_NorESM + Oth_forcings_CNRM + Oth_forcings_IPSL) / 6
#     Irr_forcings_all = (Irr_forcings_CESM2 + Irr_forcings_CESM2_gw + Irr_forcings_NorESM + Irr_forcings_CNRM + Irr_forcings_IPSL) / 6

def calcu_global_Water(IWW_1901_2014, AREA):
    area_for_calcu = AREA.T
    area_for_calcu[np.isnan(IWW_1901_2014[0, :, :])] = np.nan
    IWW_globe_1901_2014 = IWW_1901_2014 * AREA.T
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=0)
    
    return IWW_globe_1901_2014/area_for_calcu

def calcu_Water_region(IWW_1901_2014, AREA, region_id):
    AREA = get_data_from_mat('AREA.mat', 'AREA') # Area for CLM_data
    area_for_calcu = AREA.T
    
    area_for_calcu[np.isnan(IWW_1901_2014[0, :, :])] = np.nan
    area_for_calcu[ar6_region <= region_id-0.2] = np.nan
    area_for_calcu[ar6_region >= region_id+0.2] = np.nan
    
    IWW_globe_1901_2014 = IWW_1901_2014 * area_for_calcu
    IWW_globe_1901_2014[:, ar6_region<=region_id-0.2] = np.nan
    IWW_globe_1901_2014[:, ar6_region>=region_id+0.2] = np.nan
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    
    area_for_calcu = np.nansum(area_for_calcu, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=0)
    
    return IWW_globe_1901_2014/area_for_calcu

def plot_line(ax, index, data_mean, data_min, data_max, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=14, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.8)
    ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 16)
    plt.title(title1, loc='left', fontsize = 16)
    plt.ylabel(ylabel, fontsize = 18)
    plt.xlabel(xlabel, fontsize = 18)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2015)
    plt.legend(loc = 'upper left', fontsize = 16)

In [3]:
IWW_IRR_1901_2014_CESM2_RAIN, IWW_NOI_1901_2014_CESM2_RAIN = get_data_CESM2('RAIN_FROM_ATM')
IWW_IRR_1901_2014_CESM2_gw_RAIN, IWW_NOI_1901_2014_CESM2_gw_RAIN = get_data_CESM2_gw('RAIN_FROM_ATM')
IWW_IRR_1901_2014_NorESM_RAIN, IWW_NOI_1901_2014_NorESM_RAIN = get_data_NorESM('RAIN_FROM_ATM')
IWW_IRR_1901_2014_E3SM_RAIN, IWW_NOI_1901_2014_E3SM_RAIN = get_data_E3SM('RAIN')


IWW_IRR_1901_2014_CESM2_SNOW, IWW_NOI_1901_2014_CESM2_SNOW = get_data_CESM2('SNOW_FROM_ATM')
IWW_IRR_1901_2014_CESM2_gw_SNOW, IWW_NOI_1901_2014_CESM2_gw_SNOW = get_data_CESM2_gw('SNOW_FROM_ATM')
IWW_IRR_1901_2014_NorESM_SNOW, IWW_NOI_1901_2014_NorESM_SNOW = get_data_NorESM('SNOW_FROM_ATM')
IWW_IRR_1901_2014_E3SM_SNOW, IWW_NOI_1901_2014_E3SM_SNOW = get_data_E3SM('SNOW')


IWW_IRR_1901_2014_CESM2_P = IWW_IRR_1901_2014_CESM2_RAIN + IWW_IRR_1901_2014_CESM2_SNOW
IWW_IRR_1901_2014_CESM2_gw_P = IWW_IRR_1901_2014_CESM2_gw_RAIN + IWW_IRR_1901_2014_CESM2_gw_SNOW
IWW_IRR_1901_2014_NorESM_P = IWW_IRR_1901_2014_NorESM_RAIN + IWW_IRR_1901_2014_NorESM_SNOW
IWW_IRR_1901_2014_E3SM_P = IWW_IRR_1901_2014_E3SM_RAIN + IWW_IRR_1901_2014_E3SM_SNOW

IWW_NOI_1901_2014_CESM2_P = IWW_NOI_1901_2014_CESM2_RAIN + IWW_NOI_1901_2014_CESM2_SNOW
IWW_NOI_1901_2014_CESM2_gw_P = IWW_NOI_1901_2014_CESM2_gw_RAIN + IWW_NOI_1901_2014_CESM2_gw_SNOW
IWW_NOI_1901_2014_NorESM_P = IWW_NOI_1901_2014_NorESM_RAIN + IWW_NOI_1901_2014_NorESM_SNOW
IWW_NOI_1901_2014_E3SM_P = IWW_NOI_1901_2014_E3SM_RAIN + IWW_NOI_1901_2014_E3SM_SNOW

IWW_IRR_1901_2014_IPSL_P, IWW_NOI_1901_2014_IPSL_P = get_data_IPSL('pr')
IWW_IRR_1901_2014_CNRM_P, IWW_NOI_1901_2014_CNRM_P = get_data_CNRM('pr')
IWW_IRR_1901_2014_MIROC_P, IWW_NOI_1901_2014_MIROC_P = get_data_MIROC('pr')


IWW_IRR_1901_2014_CESM2_R, IWW_NOI_1901_2014_CESM2_R = get_data_CESM2('QRUNOFF')
IWW_IRR_1901_2014_CESM2_gw_R, IWW_NOI_1901_2014_CESM2_gw_R = get_data_CESM2_gw('QRUNOFF')
IWW_IRR_1901_2014_NorESM_R, IWW_NOI_1901_2014_NorESM_R = get_data_NorESM('QRUNOFF')
IWW_IRR_1901_2014_E3SM_R, IWW_NOI_1901_2014_E3SM_R = get_data_E3SM('QRUNOFF')

IWW_IRR_1901_2014_IPSL_R, IWW_NOI_1901_2014_IPSL_R = get_data_IPSL('mrro')
IWW_IRR_1901_2014_CNRM_R, IWW_NOI_1901_2014_CNRM_R = get_data_CNRM('mrro')
IWW_IRR_1901_2014_MIROC_R, IWW_NOI_1901_2014_MIROC_R = get_data_MIROC('mrro')

IWW_IRR_1901_2014_CESM2_ET, IWW_NOI_1901_2014_CESM2_ET = get_data_CESM2('QFLX_EVAP_TOT')
IWW_IRR_1901_2014_CESM2_gw_ET, IWW_NOI_1901_2014_CESM2_gw_ET = get_data_CESM2_gw('QFLX_EVAP_TOT')
IWW_IRR_1901_2014_NorESM_ET, IWW_NOI_1901_2014_NorESM_ET = get_data_NorESM('QFLX_EVAP_TOT')

IWW_IRR_1901_2014_E3SM_E1, IWW_NOI_1901_2014_E3SM_E1 = get_data_E3SM('QSOIL')
IWW_IRR_1901_2014_E3SM_E2, IWW_NOI_1901_2014_E3SM_E2 = get_data_E3SM('QVEGE')
IWW_IRR_1901_2014_E3SM_T, IWW_NOI_1901_2014_E3SM_T = get_data_E3SM('QVEGT')

IWW_IRR_1901_2014_E3SM_ET = IWW_IRR_1901_2014_E3SM_E1 + IWW_IRR_1901_2014_E3SM_E2 + IWW_IRR_1901_2014_E3SM_T
IWW_NOI_1901_2014_E3SM_ET = IWW_NOI_1901_2014_E3SM_E1 + IWW_NOI_1901_2014_E3SM_E2 + IWW_NOI_1901_2014_E3SM_T

IWW_IRR_1901_2014_IPSL_E, IWW_NOI_1901_2014_IPSL_E = get_data_IPSL('evspsbl')
IWW_IRR_1901_2014_CNRM_E, IWW_NOI_1901_2014_CNRM_E = get_data_CNRM('evspsbl')
IWW_IRR_1901_2014_MIROC_E, IWW_NOI_1901_2014_MIROC_E = get_data_MIROC('evspsbl')

IWW_IRR_1901_2014_IPSL_T, IWW_NOI_1901_2014_IPSL_T = get_data_IPSL('tran')
IWW_IRR_1901_2014_CNRM_T, IWW_NOI_1901_2014_CNRM_T = get_data_CNRM('tran')
IWW_IRR_1901_2014_MIROC_T, IWW_NOI_1901_2014_MIROC_T = get_data_MIROC('tran')

IWW_IRR_1901_2014_IPSL_ET = IWW_IRR_1901_2014_IPSL_E
IWW_IRR_1901_2014_CNRM_ET = IWW_IRR_1901_2014_CNRM_E
IWW_IRR_1901_2014_MIROC_ET = IWW_IRR_1901_2014_MIROC_E + IWW_IRR_1901_2014_MIROC_T
IWW_NOI_1901_2014_IPSL_ET = IWW_NOI_1901_2014_IPSL_E
IWW_NOI_1901_2014_CNRM_ET = IWW_NOI_1901_2014_CNRM_E
IWW_NOI_1901_2014_MIROC_ET = IWW_NOI_1901_2014_MIROC_E + IWW_NOI_1901_2014_MIROC_T



IWW_IRR_1901_2014_CESM2_P_E = IWW_IRR_1901_2014_CESM2_P - IWW_IRR_1901_2014_CESM2_ET
IWW_NOI_1901_2014_CESM2_P_E = IWW_NOI_1901_2014_CESM2_P - IWW_NOI_1901_2014_CESM2_ET

IWW_IRR_1901_2014_CESM2_gw_P_E = IWW_IRR_1901_2014_CESM2_gw_P - IWW_IRR_1901_2014_CESM2_gw_ET
IWW_NOI_1901_2014_CESM2_gw_P_E = IWW_NOI_1901_2014_CESM2_gw_P - IWW_NOI_1901_2014_CESM2_gw_ET

IWW_IRR_1901_2014_NorESM_P_E = IWW_IRR_1901_2014_NorESM_P - IWW_IRR_1901_2014_NorESM_ET
IWW_NOI_1901_2014_NorESM_P_E = IWW_NOI_1901_2014_NorESM_P - IWW_NOI_1901_2014_NorESM_ET

IWW_IRR_1901_2014_E3SM_P_E = IWW_IRR_1901_2014_E3SM_P - IWW_IRR_1901_2014_E3SM_ET
IWW_NOI_1901_2014_E3SM_P_E = IWW_NOI_1901_2014_E3SM_P - IWW_NOI_1901_2014_E3SM_ET

IWW_IRR_1901_2014_IPSL_P_E = IWW_IRR_1901_2014_IPSL_P - IWW_IRR_1901_2014_IPSL_ET
IWW_NOI_1901_2014_IPSL_P_E = IWW_NOI_1901_2014_IPSL_P - IWW_NOI_1901_2014_IPSL_ET

IWW_IRR_1901_2014_CNRM_P_E = IWW_IRR_1901_2014_CNRM_P - IWW_IRR_1901_2014_CNRM_ET
IWW_NOI_1901_2014_CNRM_P_E = IWW_NOI_1901_2014_CNRM_P - IWW_NOI_1901_2014_CNRM_ET

IWW_IRR_1901_2014_MIROC_P_E = IWW_IRR_1901_2014_MIROC_P - IWW_IRR_1901_2014_MIROC_ET
IWW_NOI_1901_2014_MIROC_P_E = IWW_NOI_1901_2014_MIROC_P - IWW_NOI_1901_2014_MIROC_ET

In [4]:
IWW_IRR_1901_2014_CESM2_TWS, IWW_NOI_1901_2014_CESM2_TWS = get_data_CESM2('TWS')
IWW_IRR_1901_2014_CESM2_gw_TWS, IWW_NOI_1901_2014_CESM2_gw_TWS = get_data_CESM2_gw('TWS')
IWW_IRR_1901_2014_NorESM_TWS, IWW_NOI_1901_2014_NorESM_TWS = get_data_NorESM('TWS')
IWW_IRR_1901_2014_E3SM_TWS, IWW_NOI_1901_2014_E3SM_TWS = get_data_E3SM('TWS')

IWW_IRR_1901_2014_IPSL_TWS, IWW_NOI_1901_2014_IPSL_TWS = get_data_IPSL('mrtws')
IWW_IRR_1901_2014_CNRM_TWS, IWW_NOI_1901_2014_CNRM_TWS = get_data_CNRM('mrtws')

IWW_IRR_1901_2014_MIROC_TWS, IWW_NOI_1901_2014_MIROC_TWS = get_data_MIROC('tws')

In [5]:
IWW_IRR_1901_2014_E3SM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_IRR_1901_2014_IPSL_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_IRR_1901_2014_CNRM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_IRR_1901_2014_MIROC_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan

IWW_NOI_1901_2014_E3SM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_NOI_1901_2014_IPSL_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_NOI_1901_2014_CNRM_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan
IWW_NOI_1901_2014_MIROC_P[np.isnan(IWW_IRR_1901_2014_CESM2_P)] = np.nan

IWW_IRR_1901_2014_E3SM_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan
IWW_IRR_1901_2014_IPSL_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan
IWW_IRR_1901_2014_CNRM_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan
IWW_IRR_1901_2014_MIROC_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan

IWW_NOI_1901_2014_E3SM_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan
IWW_NOI_1901_2014_IPSL_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan
IWW_NOI_1901_2014_CNRM_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan
IWW_NOI_1901_2014_MIROC_TWS[np.isnan(IWW_IRR_1901_2014_CESM2_TWS)] = np.nan

IWW_IRR_1901_2014_E3SM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_IPSL_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_CNRM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_MIROC_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan

IWW_NOI_1901_2014_E3SM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_NOI_1901_2014_IPSL_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_NOI_1901_2014_CNRM_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_NOI_1901_2014_MIROC_ET[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan

IWW_IRR_1901_2014_E3SM_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_IPSL_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_CNRM_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan
IWW_IRR_1901_2014_MIROC_R[np.isnan(IWW_IRR_1901_2014_CESM2_ET)] = np.nan

IWW_NOI_1901_2014_E3SM_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan
IWW_NOI_1901_2014_IPSL_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan
IWW_NOI_1901_2014_CNRM_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan
IWW_NOI_1901_2014_MIROC_R[np.isnan(IWW_IRR_1901_2014_CESM2_R)] = np.nan

In [6]:
def get_regional_P_E(region_id):
    IWW_IRR_1901_2014_CESM2_P_E_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_P_E, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_P_E_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_P_E, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_P_E_global = IWW_IRR_1901_2014_CESM2_P_E_global - np.mean(IWW_NOI_1901_2014_CESM2_P_E_global[0:114])
    IWW_NOI_1901_2014_CESM2_P_E_global = IWW_NOI_1901_2014_CESM2_P_E_global - np.mean(IWW_NOI_1901_2014_CESM2_P_E_global[0:114])

    IWW_IRR_1901_2014_CESM2_gw_P_E_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_P_E, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_P_E_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_P_E, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_gw_P_E_global = IWW_IRR_1901_2014_CESM2_gw_P_E_global - np.mean(IWW_NOI_1901_2014_CESM2_gw_P_E_global[0:114])
    IWW_NOI_1901_2014_CESM2_gw_P_E_global = IWW_NOI_1901_2014_CESM2_gw_P_E_global - np.mean(IWW_NOI_1901_2014_CESM2_gw_P_E_global[0:114])

    IWW_IRR_1901_2014_NorESM_P_E_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_P_E, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_P_E_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_P_E, AREA, region_id)
    IWW_IRR_1901_2014_NorESM_P_E_global = IWW_IRR_1901_2014_NorESM_P_E_global - np.mean(IWW_NOI_1901_2014_NorESM_P_E_global[0:114])
    IWW_NOI_1901_2014_NorESM_P_E_global = IWW_NOI_1901_2014_NorESM_P_E_global - np.mean(IWW_NOI_1901_2014_NorESM_P_E_global[0:114])

    IWW_IRR_1901_2014_E3SM_P_E_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_P_E, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_P_E_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_P_E, AREA, region_id)
    IWW_IRR_1901_2014_E3SM_P_E_global = IWW_IRR_1901_2014_E3SM_P_E_global - np.mean(IWW_NOI_1901_2014_E3SM_P_E_global[0:114])
    IWW_NOI_1901_2014_E3SM_P_E_global = IWW_NOI_1901_2014_E3SM_P_E_global - np.mean(IWW_NOI_1901_2014_E3SM_P_E_global[0:114])

    IWW_IRR_1901_2014_IPSL_P_E_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_P_E, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_P_E_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_P_E, AREA, region_id)
    IWW_IRR_1901_2014_IPSL_P_E_global = IWW_IRR_1901_2014_IPSL_P_E_global - np.mean(IWW_NOI_1901_2014_IPSL_P_E_global[0:114])
    IWW_NOI_1901_2014_IPSL_P_E_global = IWW_NOI_1901_2014_IPSL_P_E_global - np.mean(IWW_NOI_1901_2014_IPSL_P_E_global[0:114])


    IWW_IRR_1901_2014_CNRM_P_E_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_P_E, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_P_E_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_P_E, AREA, region_id)
    IWW_IRR_1901_2014_CNRM_P_E_global = IWW_IRR_1901_2014_CNRM_P_E_global - np.mean(IWW_NOI_1901_2014_CNRM_P_E_global[0:114])
    IWW_NOI_1901_2014_CNRM_P_E_global = IWW_NOI_1901_2014_CNRM_P_E_global - np.mean(IWW_NOI_1901_2014_CNRM_P_E_global[0:114])

    
    IWW_IRR_1901_2014_MIROC_P_E_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_P_E, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_P_E_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_P_E, AREA, region_id)
    IWW_IRR_1901_2014_MIROC_P_E_global = IWW_IRR_1901_2014_MIROC_P_E_global - np.mean(IWW_NOI_1901_2014_MIROC_P_E_global[0:114])
    IWW_NOI_1901_2014_MIROC_P_E_global = IWW_NOI_1901_2014_MIROC_P_E_global - np.mean(IWW_NOI_1901_2014_MIROC_P_E_global[0:114])

    
    IWW_IRR_1901_2014_all_P_E = np.vstack((IWW_IRR_1901_2014_CESM2_P_E_global,
                                   IWW_IRR_1901_2014_CESM2_gw_P_E_global,
                                   IWW_IRR_1901_2014_NorESM_P_E_global,
                                   IWW_IRR_1901_2014_E3SM_P_E_global,
                                   IWW_IRR_1901_2014_IPSL_P_E_global,
                                   IWW_IRR_1901_2014_MIROC_P_E_global,
                                   IWW_IRR_1901_2014_CNRM_P_E_global
                                          ))

    IWW_NOI_1901_2014_all_P_E = np.vstack((IWW_NOI_1901_2014_CESM2_P_E_global,
                                   IWW_NOI_1901_2014_CESM2_gw_P_E_global,
                                   IWW_NOI_1901_2014_NorESM_P_E_global,
                                   IWW_NOI_1901_2014_E3SM_P_E_global,
                                   IWW_NOI_1901_2014_IPSL_P_E_global,
                                   IWW_NOI_1901_2014_MIROC_P_E_global,
                                   IWW_NOI_1901_2014_CNRM_P_E_global
                                          ))
    
    IWW_IRR_NOI_1901_2014_all_P_E = IWW_IRR_1901_2014_all_P_E - IWW_NOI_1901_2014_all_P_E
    
    return IWW_IRR_1901_2014_all_P_E, IWW_NOI_1901_2014_all_P_E, IWW_IRR_NOI_1901_2014_all_P_E

In [7]:
IWW_IRR_1901_2014_all_P_E_SAS, IWW_NOI_1901_2014_all_P_E_SAS, IWW_IRR_NOI_1901_2014_all_P_E_SAS = get_regional_P_E(38)
IWW_IRR_1901_2014_all_P_E_SAS_75 = np.percentile(IWW_IRR_1901_2014_all_P_E_SAS, 75, axis = 0)
IWW_IRR_1901_2014_all_P_E_SAS_50 = np.percentile(IWW_IRR_1901_2014_all_P_E_SAS, 50, axis = 0)
IWW_IRR_1901_2014_all_P_E_SAS_25 = np.percentile(IWW_IRR_1901_2014_all_P_E_SAS, 25, axis = 0)
IWW_IRR_1901_2014_all_P_E_SAS_mean = np.mean(IWW_IRR_1901_2014_all_P_E_SAS, axis = 0)

IWW_NOI_1901_2014_all_P_E_SAS_75 = np.percentile(IWW_NOI_1901_2014_all_P_E_SAS, 75, axis = 0)
IWW_NOI_1901_2014_all_P_E_SAS_50 = np.percentile(IWW_NOI_1901_2014_all_P_E_SAS, 50, axis = 0)
IWW_NOI_1901_2014_all_P_E_SAS_25 = np.percentile(IWW_NOI_1901_2014_all_P_E_SAS, 25, axis = 0)
IWW_NOI_1901_2014_all_P_E_SAS_mean = np.mean(IWW_NOI_1901_2014_all_P_E_SAS, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_E_SAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_SAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_SAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_SAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_SAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_SAS, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_SAS_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_E_SAS, axis = 0)



IWW_IRR_1901_2014_all_P_E_EAS, IWW_NOI_1901_2014_all_P_E_EAS, IWW_IRR_NOI_1901_2014_all_P_E_EAS = get_regional_P_E(36)
IWW_IRR_1901_2014_all_P_E_EAS_75 = np.percentile(IWW_IRR_1901_2014_all_P_E_EAS, 75, axis = 0)
IWW_IRR_1901_2014_all_P_E_EAS_50 = np.percentile(IWW_IRR_1901_2014_all_P_E_EAS, 50, axis = 0)
IWW_IRR_1901_2014_all_P_E_EAS_25 = np.percentile(IWW_IRR_1901_2014_all_P_E_EAS, 25, axis = 0)

IWW_NOI_1901_2014_all_P_E_EAS_75 = np.percentile(IWW_NOI_1901_2014_all_P_E_EAS, 75, axis = 0)
IWW_NOI_1901_2014_all_P_E_EAS_50 = np.percentile(IWW_NOI_1901_2014_all_P_E_EAS, 50, axis = 0)
IWW_NOI_1901_2014_all_P_E_EAS_25 = np.percentile(IWW_NOI_1901_2014_all_P_E_EAS, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_E_EAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_EAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_EAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_EAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_EAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_EAS, 25, axis = 0)



IWW_IRR_1901_2014_all_P_E_MED, IWW_NOI_1901_2014_all_P_E_MED, IWW_IRR_NOI_1901_2014_all_P_E_MED = get_regional_P_E(20)
IWW_IRR_1901_2014_all_P_E_MED_75 = np.percentile(IWW_IRR_1901_2014_all_P_E_MED, 75, axis = 0)
IWW_IRR_1901_2014_all_P_E_MED_50 = np.percentile(IWW_IRR_1901_2014_all_P_E_MED, 50, axis = 0)
IWW_IRR_1901_2014_all_P_E_MED_25 = np.percentile(IWW_IRR_1901_2014_all_P_E_MED, 25, axis = 0)
IWW_IRR_1901_2014_all_P_E_MED_mean = np.mean(IWW_IRR_1901_2014_all_P_E_MED, axis = 0)

IWW_NOI_1901_2014_all_P_E_MED_75 = np.percentile(IWW_NOI_1901_2014_all_P_E_MED, 75, axis = 0)
IWW_NOI_1901_2014_all_P_E_MED_50 = np.percentile(IWW_NOI_1901_2014_all_P_E_MED, 50, axis = 0)
IWW_NOI_1901_2014_all_P_E_MED_25 = np.percentile(IWW_NOI_1901_2014_all_P_E_MED, 25, axis = 0)
IWW_NOI_1901_2014_all_P_E_MED_mean = np.mean(IWW_NOI_1901_2014_all_P_E_MED, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_E_MED_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_MED, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_MED_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_MED, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_MED_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_MED, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_MED_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_E_MED, axis = 0)



IWW_IRR_1901_2014_all_P_E_WNA, IWW_NOI_1901_2014_all_P_E_WNA, IWW_IRR_NOI_1901_2014_all_P_E_WNA = get_regional_P_E(4)
IWW_IRR_1901_2014_all_P_E_WNA_75 = np.percentile(IWW_IRR_1901_2014_all_P_E_WNA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_E_WNA_50 = np.percentile(IWW_IRR_1901_2014_all_P_E_WNA, 50, axis = 0)
IWW_IRR_1901_2014_all_P_E_WNA_25 = np.percentile(IWW_IRR_1901_2014_all_P_E_WNA, 25, axis = 0)

IWW_NOI_1901_2014_all_P_E_WNA_75 = np.percentile(IWW_NOI_1901_2014_all_P_E_WNA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_E_WNA_50 = np.percentile(IWW_NOI_1901_2014_all_P_E_WNA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_E_WNA_25 = np.percentile(IWW_NOI_1901_2014_all_P_E_WNA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_E_WNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_WNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_WNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_WNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_WNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_WNA, 25, axis = 0)



IWW_IRR_1901_2014_all_P_E_CNA, IWW_NOI_1901_2014_all_P_E_CNA, IWW_IRR_NOI_1901_2014_all_P_E_CNA = get_regional_P_E(5)
IWW_IRR_1901_2014_all_P_E_CNA_75 = np.percentile(IWW_IRR_1901_2014_all_P_E_CNA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_E_CNA_50 = np.percentile(IWW_IRR_1901_2014_all_P_E_CNA, 50, axis = 0)
IWW_IRR_1901_2014_all_P_E_CNA_25 = np.percentile(IWW_IRR_1901_2014_all_P_E_CNA, 25, axis = 0)
IWW_IRR_1901_2014_all_P_E_CNA_mean = np.mean(IWW_IRR_1901_2014_all_P_E_CNA, axis = 0)

IWW_NOI_1901_2014_all_P_E_CNA_75 = np.percentile(IWW_NOI_1901_2014_all_P_E_CNA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_E_CNA_50 = np.percentile(IWW_NOI_1901_2014_all_P_E_CNA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_E_CNA_25 = np.percentile(IWW_NOI_1901_2014_all_P_E_CNA, 25, axis = 0)
IWW_NOI_1901_2014_all_P_E_CNA_mean = np.mean(IWW_NOI_1901_2014_all_P_E_CNA, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_E_CNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_CNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_CNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_CNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_CNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_CNA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_CNA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_E_CNA, axis = 0)



IWW_IRR_1901_2014_all_P_E_WCA, IWW_NOI_1901_2014_all_P_E_WCA, IWW_IRR_NOI_1901_2014_all_P_E_WCA = get_regional_P_E(33)
IWW_IRR_1901_2014_all_P_E_WCA_75 = np.percentile(IWW_IRR_1901_2014_all_P_E_WCA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_E_WCA_50 = np.percentile(IWW_IRR_1901_2014_all_P_E_WCA, 50, axis = 0)
IWW_IRR_1901_2014_all_P_E_WCA_25 = np.percentile(IWW_IRR_1901_2014_all_P_E_WCA, 25, axis = 0)
IWW_IRR_1901_2014_all_P_E_WCA_mean = np.mean(IWW_IRR_1901_2014_all_P_E_WCA, axis = 0)

IWW_NOI_1901_2014_all_P_E_WCA_75 = np.percentile(IWW_NOI_1901_2014_all_P_E_WCA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_E_WCA_50 = np.percentile(IWW_NOI_1901_2014_all_P_E_WCA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_E_WCA_25 = np.percentile(IWW_NOI_1901_2014_all_P_E_WCA, 25, axis = 0)
IWW_NOI_1901_2014_all_P_E_WCA_mean = np.mean(IWW_NOI_1901_2014_all_P_E_WCA, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_E_WCA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_WCA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_WCA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_WCA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_WCA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_WCA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_WCA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_P_E_WCA, axis = 0)



IWW_IRR_1901_2014_all_P_E_SEA, IWW_NOI_1901_2014_all_P_E_SEA, IWW_IRR_NOI_1901_2014_all_P_E_SEA = get_regional_P_E(39)
IWW_IRR_1901_2014_all_P_E_SEA_75 = np.percentile(IWW_IRR_1901_2014_all_P_E_SEA, 75, axis = 0)
IWW_IRR_1901_2014_all_P_E_SEA_50 = np.percentile(IWW_IRR_1901_2014_all_P_E_SEA, 50, axis = 0)
IWW_IRR_1901_2014_all_P_E_SEA_25 = np.percentile(IWW_IRR_1901_2014_all_P_E_SEA, 25, axis = 0)

IWW_NOI_1901_2014_all_P_E_SEA_75 = np.percentile(IWW_NOI_1901_2014_all_P_E_SEA, 75, axis = 0)
IWW_NOI_1901_2014_all_P_E_SEA_50 = np.percentile(IWW_NOI_1901_2014_all_P_E_SEA, 50, axis = 0)
IWW_NOI_1901_2014_all_P_E_SEA_25 = np.percentile(IWW_NOI_1901_2014_all_P_E_SEA, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_P_E_SEA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_SEA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_SEA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_SEA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_P_E_SEA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_P_E_SEA, 25, axis = 0)


In [8]:
def plot_line(ax, index, data_mean, data_min, data_max, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.8)
    ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)
    plt.legend(loc = 'best', fontsize = 16)

In [27]:
params_1 = linregress(np.array(range(1901,1977)),savgol_filter((IWW_IRR_1901_2014_all_P_E_SAS_50[:76]),19,2))
params_2 = linregress(np.array(range(1901,1977)),savgol_filter((IWW_NOI_1901_2014_all_P_E_SAS_50[:76]),19,2))

In [31]:
params_1 = linregress(np.array(range(1977,2015)),savgol_filter((IWW_IRR_1901_2014_all_P_E_SAS_50[76:]),19,2))
params_2 = linregress(np.array(range(1977,2015)),savgol_filter((IWW_NOI_1901_2014_all_P_E_SAS_50[76:]),19,2))

In [50]:
params_1 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_SAS_50),19,2))

In [87]:
params_1 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_IRR_1901_2014_all_P_E_WCA_50),19,2))
params_2 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_NOI_1901_2014_all_P_E_WCA_50),19,2))

In [90]:
params_1 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_WCA_50),19,2))

In [69]:
params_1 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_IRR_1901_2014_all_P_E_MED_50),19,2))
params_2 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_NOI_1901_2014_all_P_E_MED_50),19,2))

In [76]:
params_1 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_MED_50),19,2))

In [79]:
params_1 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_IRR_1901_2014_all_P_E_CNA_50),19,2))
params_2 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_NOI_1901_2014_all_P_E_CNA_50),19,2))

In [84]:
params_1 = linregress(np.array(range(1901,2015)),savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_CNA_50),19,2))

In [ ]:
fig = plt.figure(figsize=(12, 20),dpi=300)
fig.subplots_adjust(hspace=0.2, wspace=0.3, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)

ax1 = plt.subplot(421, frameon=True)
index = 'a'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_P_E_SAS_50),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_SAS_25),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_SAS_75),19,2), r'South Asia', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_SAS_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_SAS_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_SAS_75),19,2), r'South Asia', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')

x1 = np.array(range(1901, 1976))
y1_irr = 1.0352630757201644 * x1 - 2032.8658182246897
y1_noi = 1.5791457488095462 * x1 -3062.07728167478
x2 = np.array(range(1976, 2014))
y2_irr = -1.7983755236511785 * x2 + 3559.736847543501
y2_noi = -0.15882463925413257 * x2 + 324.3074677583668

plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[30], y1_irr[30]-40, '+1.04***', fontsize = 16, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]+60, '+1.58***', fontsize = 16, color = 'brown', weight='bold')
plt.legend(loc = 'best', fontsize = 16)

plt.plot(x2, y2_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x2[10], y2_irr[10]-70, '-1.80***', fontsize = 16, color = 'dodgerblue', weight='bold')
plt.plot(x2, y2_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x2[15], y2_noi[15]+20, '-0.16', fontsize = 16, color = 'brown', weight='bold')




plt.grid(linestyle = '--')
plt.ylim([-250, 250])

x1 = np.array(range(1901, 2015))
y1_irr_noi = -0.6063207952233699 * x1 + 1158.781154989394

ax1 = plt.subplot(422, frameon=True)
index = 'b'
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_SAS_50),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_SAS_25),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_SAS_75),19,2), r'South Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')
plt.grid(linestyle = '--')
plt.plot(x1, y1_irr_noi, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[60], y1_irr[60]-70, '-0.61***', fontsize = 16, color = 'green', weight='bold')
plt.ylim([-200, 200])
plt.legend(loc = 'best', fontsize = 16)



ax1 = plt.subplot(423, frameon=True)
index = 'c'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_P_E_MED_50),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_MED_25),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_MED_75),19,2), r'Mediterranean', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_MED_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_MED_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_MED_75),19,2), r'Mediterranean', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')

x1 = np.array(range(1901, 2015))
y1_irr = -0.21522657162778058 * x1 + 416.67436413239386
y1_noi = 0.008539068427630053 * x1 -17.10992597967792
plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[80], y1_irr[30]-30, '-0.22***', fontsize = 16, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]+10, '+0.01', fontsize = 16, color = 'brown', weight='bold')
plt.legend(loc = 'best', fontsize = 16)
plt.grid(linestyle = '--')
plt.ylim([-50, 50])

ax1 = plt.subplot(424, frameon=True)
index = 'd'
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_MED_50),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_MED_25),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_MED_75),19,2), r'Mediterranean', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')
plt.grid(linestyle = '--')
x1 = np.array(range(1901, 2015))
y1_irr_noi = -0.21057071738367203 * x1 + 409.24097882324565
plt.plot(x1, y1_irr_noi, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[60], y1_irr_noi[60]-20, '-0.21***', fontsize = 16, color = 'green', weight='bold')
plt.legend(loc = 'best', fontsize = 16)
plt.ylim([-80, 80])





ax1 = plt.subplot(425, frameon=True)
index = 'e'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_P_E_CNA_50),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_CNA_25),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_CNA_75),19,2), r'Central North America', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_CNA_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_CNA_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_CNA_75),19,2), r'Central North America', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
y1_irr = -0.323536702223421 * x1 + 630.6145415895118
y1_noi = -0.20773505153581728 * x1 + 404.9975070721852
plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[80], y1_irr[80]-30, '-0.32***', fontsize = 16, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]+20, '-0.21***', fontsize = 16, color = 'brown', weight='bold')
plt.legend(loc = 'best', fontsize = 16)
plt.grid(linestyle = '--')
plt.ylim([-100, 100])

ax1 = plt.subplot(426, frameon=True)
index = 'f'

x1 = np.array(range(1901, 2015))
y1_irr_noi = -0.05735960033151404 * x1 + 110.82236478569943
plt.plot(x1, y1_irr_noi, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[60], y1_irr_noi[60]-30, '-0.06**', fontsize = 16, color = 'green', weight='bold')
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_CNA_50),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_CNA_25),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_CNA_75),19,2), r'Central North America', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')
plt.grid(linestyle = '--')
plt.ylim([-100, 100])



ax1 = plt.subplot(427, frameon=True)
index = 'g'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_P_E_WCA_50),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_WCA_25),19,2), savgol_filter((IWW_IRR_1901_2014_all_P_E_WCA_75),19,2), r'West Central Asia', '', 'tranirr', 'dodgerblue', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_P_E_WCA_50),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_WCA_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_P_E_WCA_75),19,2), r'West Central Asia', '', '1901irr', 'brown', r'land (P-ET) ($\mathregular{mm/year}$)', '', '')
y1_irr = -0.1261889766013906 * x1 + 237.93615469595804
y1_noi = 0.004335430490288922 * x1 -9.622798808008591
plt.plot(x1, y1_irr, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr',alpha=0.6)
plt.text(x1[80], y1_irr[80]-10, '-0.13***', fontsize = 16, color = 'dodgerblue', weight='bold')
plt.plot(x1, y1_noi, color = 'k', linestyle = '-', linewidth = 2, label='trend 1901irr',alpha=0.6)
plt.text(x1[10], y1_noi[10]+10, '0.00', fontsize = 16, color = 'brown', weight='bold')
plt.grid(linestyle = '--')
plt.ylim([-50, 50])

ax1 = plt.subplot(428, frameon=True)
index = 'h'
x1 = np.array(range(1901, 2015))
y1_irr_noi = -0.15600200753120402 * x1 + 296.15447351636874
plt.plot(x1, y1_irr_noi, color = 'k', linestyle = '--', linewidth = 2, label='trend tranirr-1901irr',alpha=0.6)
plt.text(x1[60], y1_irr_noi[60]-20, '-0.16***', fontsize = 16, color = 'green', weight='bold')
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_WCA_50),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_WCA_25),19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_P_E_WCA_75),19,2), r'West Central Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$(P-ET) ($\mathregular{mm/year}$)', '', '')
plt.grid(linestyle = '--')
plt.ylim([-50, 50])




In [35]:
IWW_IRR_1901_2014_CESM2_TWS_global = calcu_global_Water(IWW_IRR_1901_2014_CESM2_TWS, AREA)
IWW_NOI_1901_2014_CESM2_TWS_global = calcu_global_Water(IWW_NOI_1901_2014_CESM2_TWS, AREA)
IWW_IRR_1901_2014_CESM2_TWS_global = IWW_IRR_1901_2014_CESM2_TWS_global - np.mean(IWW_NOI_1901_2014_CESM2_TWS_global[0:30])
IWW_NOI_1901_2014_CESM2_TWS_global = IWW_NOI_1901_2014_CESM2_TWS_global - np.mean(IWW_NOI_1901_2014_CESM2_TWS_global[0:30])

IWW_IRR_1901_2014_CESM2_gw_TWS_global = calcu_global_Water(IWW_IRR_1901_2014_CESM2_gw_TWS, AREA)
IWW_NOI_1901_2014_CESM2_gw_TWS_global = calcu_global_Water(IWW_NOI_1901_2014_CESM2_gw_TWS, AREA)
IWW_IRR_1901_2014_CESM2_gw_TWS_global = IWW_IRR_1901_2014_CESM2_gw_TWS_global - np.mean(IWW_NOI_1901_2014_CESM2_gw_TWS_global[0:30])
IWW_NOI_1901_2014_CESM2_gw_TWS_global = IWW_NOI_1901_2014_CESM2_gw_TWS_global - np.mean(IWW_NOI_1901_2014_CESM2_gw_TWS_global[0:30])

IWW_IRR_1901_2014_NorESM_TWS_global = calcu_global_Water(IWW_IRR_1901_2014_NorESM_TWS, AREA)
IWW_NOI_1901_2014_NorESM_TWS_global = calcu_global_Water(IWW_NOI_1901_2014_NorESM_TWS, AREA)
IWW_IRR_1901_2014_NorESM_TWS_global = IWW_IRR_1901_2014_NorESM_TWS_global - np.mean(IWW_NOI_1901_2014_NorESM_TWS_global[0:30])
IWW_NOI_1901_2014_NorESM_TWS_global = IWW_NOI_1901_2014_NorESM_TWS_global - np.mean(IWW_NOI_1901_2014_NorESM_TWS_global[0:30])

IWW_IRR_1901_2014_E3SM_TWS_global = calcu_global_Water(IWW_IRR_1901_2014_E3SM_TWS, AREA)
IWW_NOI_1901_2014_E3SM_TWS_global = calcu_global_Water(IWW_NOI_1901_2014_E3SM_TWS, AREA)
IWW_IRR_1901_2014_E3SM_TWS_global = IWW_IRR_1901_2014_E3SM_TWS_global - np.mean(IWW_NOI_1901_2014_E3SM_TWS_global[0:30])
IWW_NOI_1901_2014_E3SM_TWS_global = IWW_NOI_1901_2014_E3SM_TWS_global - np.mean(IWW_NOI_1901_2014_E3SM_TWS_global[0:30])

IWW_IRR_1901_2014_IPSL_TWS_global = calcu_global_Water(IWW_IRR_1901_2014_IPSL_TWS, AREA)
IWW_NOI_1901_2014_IPSL_TWS_global = calcu_global_Water(IWW_NOI_1901_2014_IPSL_TWS, AREA)
IWW_IRR_1901_2014_IPSL_TWS_global = IWW_IRR_1901_2014_IPSL_TWS_global - np.mean(IWW_NOI_1901_2014_IPSL_TWS_global[0:30])
IWW_NOI_1901_2014_IPSL_TWS_global = IWW_NOI_1901_2014_IPSL_TWS_global - np.mean(IWW_NOI_1901_2014_IPSL_TWS_global[0:30])

IWW_IRR_1901_2014_CNRM_TWS_global = calcu_global_Water(IWW_IRR_1901_2014_CNRM_TWS, AREA)
IWW_NOI_1901_2014_CNRM_TWS_global = calcu_global_Water(IWW_NOI_1901_2014_CNRM_TWS, AREA)
IWW_IRR_1901_2014_CNRM_TWS_global = IWW_IRR_1901_2014_CNRM_TWS_global - np.mean(IWW_NOI_1901_2014_CNRM_TWS_global[0:30])
IWW_NOI_1901_2014_CNRM_TWS_global = IWW_NOI_1901_2014_CNRM_TWS_global - np.mean(IWW_NOI_1901_2014_CNRM_TWS_global[0:30])

IWW_IRR_1901_2014_MIROC_TWS_global = calcu_global_Water(IWW_IRR_1901_2014_MIROC_TWS, AREA)
IWW_NOI_1901_2014_MIROC_TWS_global = calcu_global_Water(IWW_NOI_1901_2014_MIROC_TWS, AREA)
IWW_IRR_1901_2014_MIROC_TWS_global = IWW_IRR_1901_2014_MIROC_TWS_global - np.mean(IWW_NOI_1901_2014_MIROC_TWS_global[0:30])
IWW_NOI_1901_2014_MIROC_TWS_global = IWW_NOI_1901_2014_MIROC_TWS_global - np.mean(IWW_NOI_1901_2014_MIROC_TWS_global[0:30])

In [36]:
def get_regional_TWS(region_id):
    

    IWW_IRR_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_TWS_global = IWW_IRR_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_TWS_global = IWW_NOI_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]

    IWW_IRR_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_gw_TWS_global = IWW_IRR_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = IWW_NOI_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    
    IWW_IRR_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_NorESM_TWS_global = IWW_IRR_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]
    IWW_NOI_1901_2014_NorESM_TWS_global = IWW_NOI_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]

    IWW_IRR_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_E3SM_TWS_global = IWW_IRR_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]
    IWW_NOI_1901_2014_E3SM_TWS_global = IWW_NOI_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]

    IWW_IRR_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_IRR_1901_2014_IPSL_TWS_global = IWW_IRR_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]
    IWW_NOI_1901_2014_IPSL_TWS_global = IWW_NOI_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]

    IWW_IRR_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CNRM_TWS_global = IWW_IRR_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    IWW_NOI_1901_2014_CNRM_TWS_global = IWW_NOI_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    
    IWW_IRR_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_IRR_1901_2014_MIROC_TWS_global = IWW_IRR_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]
    IWW_NOI_1901_2014_MIROC_TWS_global = IWW_NOI_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]


#     IWW_IRR_1901_2014_all_TWS = np.vstack((IWW_IRR_1901_2014_CESM2_TWS_global,
#                                IWW_IRR_1901_2014_CESM2_gw_TWS_global,
#                                IWW_IRR_1901_2014_NorESM_TWS_global,
#                                IWW_IRR_1901_2014_E3SM_TWS_global,
#                                IWW_IRR_1901_2014_IPSL_TWS_global,
#                                IWW_IRR_1901_2014_CNRM_TWS_global))
    


#     IWW_NOI_1901_2014_all_TWS = np.vstack((IWW_NOI_1901_2014_CESM2_TWS_global,
#                                    IWW_NOI_1901_2014_CESM2_gw_TWS_global,
#                                    IWW_NOI_1901_2014_NorESM_TWS_global,
#                                    IWW_NOI_1901_2014_E3SM_TWS_global,
#                                    IWW_NOI_1901_2014_IPSL_TWS_global,
#                                    IWW_NOI_1901_2014_CNRM_TWS_global))
    
    IWW_IRR_1901_2014_all_TWS = np.vstack((
                               IWW_IRR_1901_2014_CESM2_gw_TWS_global,
#         IWW_IRR_1901_2014_E3SM_TWS_global,
                               
                               
                               IWW_IRR_1901_2014_MIROC_TWS_global,
                               IWW_IRR_1901_2014_CNRM_TWS_global))
    


    IWW_NOI_1901_2014_all_TWS = np.vstack((
                                   IWW_NOI_1901_2014_CESM2_gw_TWS_global,
#         IWW_NOI_1901_2014_E3SM_TWS_global,
                                   
                                   
                                   IWW_NOI_1901_2014_MIROC_TWS_global,
                                   IWW_NOI_1901_2014_CNRM_TWS_global))
    
#     IWW_IRR_1901_2014_all_TWS = np.vstack((IWW_IRR_1901_2014_CESM2_gw_TWS_global,
#                                IWW_IRR_1901_2014_E3SM_TWS_global,
#                                IWW_IRR_1901_2014_IPSL_TWS_global))
    


#     IWW_NOI_1901_2014_all_TWS = np.vstack((IWW_NOI_1901_2014_CESM2_gw_TWS_global,
#                                    IWW_NOI_1901_2014_E3SM_TWS_global,
#                                    IWW_NOI_1901_2014_IPSL_TWS_global))


    
    IWW_IRR_NOI_1901_2014_all_TWS = IWW_IRR_1901_2014_all_TWS - IWW_NOI_1901_2014_all_TWS
    
    return IWW_IRR_1901_2014_all_TWS, IWW_NOI_1901_2014_all_TWS, IWW_IRR_NOI_1901_2014_all_TWS

In [37]:
def get_regional_TWS_IPSL(region_id):
    

    IWW_IRR_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_TWS_global = IWW_IRR_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_TWS_global = IWW_NOI_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]

    IWW_IRR_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_gw_TWS_global = IWW_IRR_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = IWW_NOI_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    
    IWW_IRR_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_NorESM_TWS_global = IWW_IRR_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]
    IWW_NOI_1901_2014_NorESM_TWS_global = IWW_NOI_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]

    IWW_IRR_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_E3SM_TWS_global = IWW_IRR_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]
    IWW_NOI_1901_2014_E3SM_TWS_global = IWW_NOI_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]

    IWW_IRR_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_IRR_1901_2014_IPSL_TWS_global = IWW_IRR_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]
    IWW_NOI_1901_2014_IPSL_TWS_global = IWW_NOI_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]

    IWW_IRR_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CNRM_TWS_global = IWW_IRR_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    IWW_NOI_1901_2014_CNRM_TWS_global = IWW_NOI_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    
    IWW_IRR_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_IRR_1901_2014_MIROC_TWS_global = IWW_IRR_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]
    IWW_NOI_1901_2014_MIROC_TWS_global = IWW_NOI_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]


    IWW_IRR_1901_2014_all_TWS = np.vstack((IWW_IRR_1901_2014_IPSL_TWS_global))

    IWW_NOI_1901_2014_all_TWS = np.vstack((IWW_NOI_1901_2014_IPSL_TWS_global))

    IWW_IRR_NOI_1901_2014_all_TWS = IWW_IRR_1901_2014_all_TWS - IWW_NOI_1901_2014_all_TWS
    
    return IWW_IRR_1901_2014_all_TWS, IWW_NOI_1901_2014_all_TWS, IWW_IRR_NOI_1901_2014_all_TWS

In [38]:
def get_regional_TWS_E3SM(region_id):
    

    IWW_IRR_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_TWS_global = IWW_IRR_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_TWS_global = IWW_NOI_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]

    IWW_IRR_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_gw_TWS_global = IWW_IRR_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = IWW_NOI_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    
    IWW_IRR_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_NorESM_TWS_global = IWW_IRR_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]
    IWW_NOI_1901_2014_NorESM_TWS_global = IWW_NOI_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]

    IWW_IRR_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_E3SM_TWS_global = IWW_IRR_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]
    IWW_NOI_1901_2014_E3SM_TWS_global = IWW_NOI_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]

    IWW_IRR_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_IRR_1901_2014_IPSL_TWS_global = IWW_IRR_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]
    IWW_NOI_1901_2014_IPSL_TWS_global = IWW_NOI_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]

    IWW_IRR_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CNRM_TWS_global = IWW_IRR_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    IWW_NOI_1901_2014_CNRM_TWS_global = IWW_NOI_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    
    IWW_IRR_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_IRR_1901_2014_MIROC_TWS_global = IWW_IRR_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]
    IWW_NOI_1901_2014_MIROC_TWS_global = IWW_NOI_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]


    IWW_IRR_1901_2014_all_TWS = np.vstack((IWW_IRR_1901_2014_E3SM_TWS_global))

    IWW_NOI_1901_2014_all_TWS = np.vstack((IWW_NOI_1901_2014_E3SM_TWS_global))

    IWW_IRR_NOI_1901_2014_all_TWS = IWW_IRR_1901_2014_all_TWS - IWW_NOI_1901_2014_all_TWS
    
    return IWW_IRR_1901_2014_all_TWS, IWW_NOI_1901_2014_all_TWS, IWW_IRR_NOI_1901_2014_all_TWS

In [39]:
def get_regional_TWS_no_grd(region_id):
    

    IWW_IRR_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_TWS_global = IWW_IRR_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_TWS_global = IWW_NOI_1901_2014_CESM2_TWS_global - IWW_NOI_1901_2014_CESM2_TWS_global[0]

    IWW_IRR_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CESM2_gw_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CESM2_gw_TWS_global = IWW_IRR_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    IWW_NOI_1901_2014_CESM2_gw_TWS_global = IWW_NOI_1901_2014_CESM2_gw_TWS_global - IWW_NOI_1901_2014_CESM2_gw_TWS_global[0]
    
    IWW_IRR_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_NorESM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_NorESM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_NorESM_TWS_global = IWW_IRR_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]
    IWW_NOI_1901_2014_NorESM_TWS_global = IWW_NOI_1901_2014_NorESM_TWS_global - IWW_NOI_1901_2014_NorESM_TWS_global[0]

    IWW_IRR_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_E3SM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_E3SM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_E3SM_TWS_global = IWW_IRR_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]
    IWW_NOI_1901_2014_E3SM_TWS_global = IWW_NOI_1901_2014_E3SM_TWS_global - IWW_NOI_1901_2014_E3SM_TWS_global[0]

    IWW_IRR_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_NOI_1901_2014_IPSL_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_IPSL_TWS, AREA, region_id)
    IWW_IRR_1901_2014_IPSL_TWS_global = IWW_IRR_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]
    IWW_NOI_1901_2014_IPSL_TWS_global = IWW_NOI_1901_2014_IPSL_TWS_global - IWW_NOI_1901_2014_IPSL_TWS_global[0]

    IWW_IRR_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_NOI_1901_2014_CNRM_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_CNRM_TWS, AREA, region_id)
    IWW_IRR_1901_2014_CNRM_TWS_global = IWW_IRR_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    IWW_NOI_1901_2014_CNRM_TWS_global = IWW_NOI_1901_2014_CNRM_TWS_global - IWW_NOI_1901_2014_CNRM_TWS_global[0]
    
    IWW_IRR_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_IRR_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_NOI_1901_2014_MIROC_TWS_global = calcu_Water_region(IWW_NOI_1901_2014_MIROC_TWS, AREA, region_id)
    IWW_IRR_1901_2014_MIROC_TWS_global = IWW_IRR_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]
    IWW_NOI_1901_2014_MIROC_TWS_global = IWW_NOI_1901_2014_MIROC_TWS_global - IWW_NOI_1901_2014_MIROC_TWS_global[0]


    IWW_IRR_1901_2014_all_TWS = np.vstack((IWW_IRR_1901_2014_CESM2_TWS_global,
                               
                               IWW_IRR_1901_2014_NorESM_TWS_global,
                               
                               ))
    


    IWW_NOI_1901_2014_all_TWS = np.vstack((IWW_NOI_1901_2014_CESM2_TWS_global,
                                   
                                   IWW_NOI_1901_2014_NorESM_TWS_global,
                                   ))



    
    IWW_IRR_NOI_1901_2014_all_TWS = IWW_IRR_1901_2014_all_TWS - IWW_NOI_1901_2014_all_TWS
    
    return IWW_IRR_1901_2014_all_TWS, IWW_NOI_1901_2014_all_TWS, IWW_IRR_NOI_1901_2014_all_TWS

In [45]:
IWW_IRR_1901_2014_all_TWS_SAS, IWW_NOI_1901_2014_all_TWS_SAS, IWW_IRR_NOI_1901_2014_all_TWS_SAS = get_regional_TWS(38)
IWW_IRR_1901_2014_all_TWS_SAS_75 = np.percentile(IWW_IRR_1901_2014_all_TWS_SAS, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_SAS_50 = np.percentile(IWW_IRR_1901_2014_all_TWS_SAS, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_SAS_25 = np.percentile(IWW_IRR_1901_2014_all_TWS_SAS, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_SAS_mean = np.mean(IWW_IRR_1901_2014_all_TWS_SAS, axis = 0)

IWW_NOI_1901_2014_all_TWS_SAS_75 = np.percentile(IWW_NOI_1901_2014_all_TWS_SAS, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_SAS_50 = np.percentile(IWW_NOI_1901_2014_all_TWS_SAS, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_SAS_25 = np.percentile(IWW_NOI_1901_2014_all_TWS_SAS, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_SAS_mean = np.mean(IWW_NOI_1901_2014_all_TWS_SAS, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_SAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_SAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_SAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_SAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_SAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_SAS, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_SAS, axis = 0)


IWW_IRR_1901_2014_all_TWS_MED, IWW_NOI_1901_2014_all_TWS_MED, IWW_IRR_NOI_1901_2014_all_TWS_MED = get_regional_TWS(20)
IWW_IRR_1901_2014_all_TWS_MED_75 = np.percentile(IWW_IRR_1901_2014_all_TWS_MED, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_MED_50 = np.percentile(IWW_IRR_1901_2014_all_TWS_MED, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_MED_25 = np.percentile(IWW_IRR_1901_2014_all_TWS_MED, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_MED_mean = np.mean(IWW_IRR_1901_2014_all_TWS_MED, axis = 0)

IWW_NOI_1901_2014_all_TWS_MED_75 = np.percentile(IWW_NOI_1901_2014_all_TWS_MED, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_MED_50 = np.percentile(IWW_NOI_1901_2014_all_TWS_MED, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_MED_25 = np.percentile(IWW_NOI_1901_2014_all_TWS_MED, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_MED_mean = np.mean(IWW_NOI_1901_2014_all_TWS_MED, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_MED_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_MED, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_MED_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_MED, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_MED_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_MED, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_MED_mean = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_MED, axis = 0)


IWW_IRR_1901_2014_all_TWS_CNA, IWW_NOI_1901_2014_all_TWS_CNA, IWW_IRR_NOI_1901_2014_all_TWS_CNA = get_regional_TWS(5)
IWW_IRR_1901_2014_all_TWS_CNA_75 = np.percentile(IWW_IRR_1901_2014_all_TWS_CNA, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_CNA_50 = np.percentile(IWW_IRR_1901_2014_all_TWS_CNA, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_CNA_25 = np.percentile(IWW_IRR_1901_2014_all_TWS_CNA, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_CNA_mean = np.mean(IWW_IRR_1901_2014_all_TWS_CNA, axis = 0)

IWW_NOI_1901_2014_all_TWS_CNA_75 = np.percentile(IWW_NOI_1901_2014_all_TWS_CNA, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_CNA_50 = np.percentile(IWW_NOI_1901_2014_all_TWS_CNA, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_CNA_25 = np.percentile(IWW_NOI_1901_2014_all_TWS_CNA, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_CNA_mean = np.mean(IWW_NOI_1901_2014_all_TWS_CNA, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_CNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_CNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_CNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_CNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_CNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_CNA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_CNA, axis = 0)


IWW_IRR_1901_2014_all_TWS_NCA, IWW_NOI_1901_2014_all_TWS_NCA, IWW_IRR_NOI_1901_2014_all_TWS_NCA = get_regional_TWS(33)
IWW_IRR_1901_2014_all_TWS_NCA_75 = np.percentile(IWW_IRR_1901_2014_all_TWS_NCA, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_NCA_50 = np.percentile(IWW_IRR_1901_2014_all_TWS_NCA, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_NCA_25 = np.percentile(IWW_IRR_1901_2014_all_TWS_NCA, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_NCA_mean = np.mean(IWW_IRR_1901_2014_all_TWS_NCA, axis = 0)

IWW_NOI_1901_2014_all_TWS_NCA_75 = np.percentile(IWW_NOI_1901_2014_all_TWS_NCA, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_NCA_50 = np.percentile(IWW_NOI_1901_2014_all_TWS_NCA, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_NCA_25 = np.percentile(IWW_NOI_1901_2014_all_TWS_NCA, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_NCA_mean = np.mean(IWW_NOI_1901_2014_all_TWS_NCA, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_NCA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_NCA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_NCA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_NCA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_NCA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_NCA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_NCA, axis = 0)


In [46]:
IWW_IRR_1901_2014_all_TWS_SAS, IWW_NOI_1901_2014_all_TWS_SAS, IWW_IRR_NOI_1901_2014_all_TWS_SAS = get_regional_TWS_no_grd(38)
IWW_IRR_1901_2014_all_TWS_SAS_75_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_SAS, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_SAS_50_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_SAS, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_SAS_25_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_SAS, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_SAS_mean_no_grd = np.mean(IWW_IRR_1901_2014_all_TWS_SAS, axis = 0)

IWW_NOI_1901_2014_all_TWS_SAS_75_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_SAS, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_SAS_50_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_SAS, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_SAS_25_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_SAS, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_SAS_mean_no_grd = np.mean(IWW_NOI_1901_2014_all_TWS_SAS, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_SAS_75_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_SAS, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_SAS_50_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_SAS, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_SAS_25_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_SAS, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean_no_grd = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_SAS, axis = 0)


IWW_IRR_1901_2014_all_TWS_MED, IWW_NOI_1901_2014_all_TWS_MED, IWW_IRR_NOI_1901_2014_all_TWS_MED = get_regional_TWS_no_grd(20)
IWW_IRR_1901_2014_all_TWS_MED_75_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_MED, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_MED_50_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_MED, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_MED_25_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_MED, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_MED_mean_no_grd = np.mean(IWW_IRR_1901_2014_all_TWS_MED, axis = 0)

IWW_NOI_1901_2014_all_TWS_MED_75_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_MED, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_MED_50_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_MED, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_MED_25_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_MED, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_MED_mean_no_grd = np.mean(IWW_NOI_1901_2014_all_TWS_MED, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_MED_75_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_MED, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_MED_50_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_MED, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_MED_25_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_MED, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_MED_mean_no_grd = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_MED, axis = 0)


IWW_IRR_1901_2014_all_TWS_CNA, IWW_NOI_1901_2014_all_TWS_CNA, IWW_IRR_NOI_1901_2014_all_TWS_CNA = get_regional_TWS_no_grd(5)
IWW_IRR_1901_2014_all_TWS_CNA_75_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_CNA, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_CNA_50_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_CNA, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_CNA_25_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_CNA, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_CNA_mean_no_grd = np.mean(IWW_IRR_1901_2014_all_TWS_CNA, axis = 0)

IWW_NOI_1901_2014_all_TWS_CNA_75_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_CNA, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_CNA_50_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_CNA, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_CNA_25_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_CNA, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_CNA_mean_no_grd = np.mean(IWW_NOI_1901_2014_all_TWS_CNA, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_CNA_75_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_CNA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_CNA_50_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_CNA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_CNA_25_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_CNA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean_no_grd = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_CNA, axis = 0)


IWW_IRR_1901_2014_all_TWS_NCA, IWW_NOI_1901_2014_all_TWS_NCA, IWW_IRR_NOI_1901_2014_all_TWS_NCA = get_regional_TWS_no_grd(33)
IWW_IRR_1901_2014_all_TWS_NCA_75_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_NCA, 75, axis = 0)
IWW_IRR_1901_2014_all_TWS_NCA_50_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_NCA, 50, axis = 0)
IWW_IRR_1901_2014_all_TWS_NCA_25_no_grd = np.percentile(IWW_IRR_1901_2014_all_TWS_NCA, 25, axis = 0)
IWW_IRR_1901_2014_all_TWS_NCA_mean_no_grd = np.mean(IWW_IRR_1901_2014_all_TWS_NCA, axis = 0)

IWW_NOI_1901_2014_all_TWS_NCA_75_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_NCA, 75, axis = 0)
IWW_NOI_1901_2014_all_TWS_NCA_50_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_NCA, 50, axis = 0)
IWW_NOI_1901_2014_all_TWS_NCA_25_no_grd = np.percentile(IWW_NOI_1901_2014_all_TWS_NCA, 25, axis = 0)
IWW_NOI_1901_2014_all_TWS_NCA_mean_no_grd = np.mean(IWW_NOI_1901_2014_all_TWS_NCA, axis = 0)

IWW_IRR_NOI_1901_2014_all_TWS_NCA_75_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_NCA, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_NCA_50_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_NCA, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_NCA_25_no_grd = np.percentile(IWW_IRR_NOI_1901_2014_all_TWS_NCA, 25, axis = 0)
IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean_no_grd = np.mean(IWW_IRR_NOI_1901_2014_all_TWS_NCA, axis = 0)


In [47]:
IWW_IRR_1901_2014_all_TWS_SAS, IWW_NOI_1901_2014_all_TWS_SAS, IWW_IRR_NOI_1901_2014_all_TWS_SAS = get_regional_TWS_E3SM(38)

IWW_IRR_1901_2014_all_TWS_SAS_mean_E3SM = np.squeeze(IWW_IRR_1901_2014_all_TWS_SAS)


IWW_NOI_1901_2014_all_TWS_SAS_mean_E3SM = np.squeeze(IWW_NOI_1901_2014_all_TWS_SAS)


IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean_E3SM = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_SAS)


IWW_IRR_1901_2014_all_TWS_MED, IWW_NOI_1901_2014_all_TWS_MED, IWW_IRR_NOI_1901_2014_all_TWS_MED = get_regional_TWS_E3SM(20)

IWW_IRR_1901_2014_all_TWS_MED_mean_E3SM = np.squeeze(IWW_IRR_1901_2014_all_TWS_MED)


IWW_NOI_1901_2014_all_TWS_MED_mean_E3SM = np.squeeze(IWW_NOI_1901_2014_all_TWS_MED)


IWW_IRR_NOI_1901_2014_all_TWS_MED_mean_E3SM = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_MED)


IWW_IRR_1901_2014_all_TWS_CNA, IWW_NOI_1901_2014_all_TWS_CNA, IWW_IRR_NOI_1901_2014_all_TWS_CNA = get_regional_TWS_E3SM(5)

IWW_IRR_1901_2014_all_TWS_CNA_mean_E3SM = np.squeeze(IWW_IRR_1901_2014_all_TWS_CNA)


IWW_NOI_1901_2014_all_TWS_CNA_mean_E3SM = np.squeeze(IWW_NOI_1901_2014_all_TWS_CNA)


IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean_E3SM = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_CNA)


IWW_IRR_1901_2014_all_TWS_NCA, IWW_NOI_1901_2014_all_TWS_NCA, IWW_IRR_NOI_1901_2014_all_TWS_NCA = get_regional_TWS_E3SM(33)

IWW_IRR_1901_2014_all_TWS_NCA_mean_E3SM = np.squeeze(IWW_IRR_1901_2014_all_TWS_NCA)


IWW_NOI_1901_2014_all_TWS_NCA_mean_E3SM = np.squeeze(IWW_NOI_1901_2014_all_TWS_NCA)


IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean_E3SM = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_NCA)


In [49]:
IWW_IRR_1901_2014_all_TWS_SAS, IWW_NOI_1901_2014_all_TWS_SAS, IWW_IRR_NOI_1901_2014_all_TWS_SAS = get_regional_TWS_IPSL(38)

IWW_IRR_1901_2014_all_TWS_SAS_mean_IPSL = np.squeeze(IWW_IRR_1901_2014_all_TWS_SAS)


IWW_NOI_1901_2014_all_TWS_SAS_mean_IPSL = np.squeeze(IWW_NOI_1901_2014_all_TWS_SAS)


IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean_IPSL = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_SAS)


IWW_IRR_1901_2014_all_TWS_MED, IWW_NOI_1901_2014_all_TWS_MED, IWW_IRR_NOI_1901_2014_all_TWS_MED = get_regional_TWS_IPSL(20)

IWW_IRR_1901_2014_all_TWS_MED_mean_IPSL = np.squeeze(IWW_IRR_1901_2014_all_TWS_MED)


IWW_NOI_1901_2014_all_TWS_MED_mean_IPSL = np.squeeze(IWW_NOI_1901_2014_all_TWS_MED)


IWW_IRR_NOI_1901_2014_all_TWS_MED_mean_IPSL = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_MED)


IWW_IRR_1901_2014_all_TWS_CNA, IWW_NOI_1901_2014_all_TWS_CNA, IWW_IRR_NOI_1901_2014_all_TWS_CNA = get_regional_TWS_IPSL(5)

IWW_IRR_1901_2014_all_TWS_CNA_mean_IPSL = np.squeeze(IWW_IRR_1901_2014_all_TWS_CNA)


IWW_NOI_1901_2014_all_TWS_CNA_mean_IPSL = np.squeeze(IWW_NOI_1901_2014_all_TWS_CNA)


IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean_IPSL = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_CNA)


IWW_IRR_1901_2014_all_TWS_NCA, IWW_NOI_1901_2014_all_TWS_NCA, IWW_IRR_NOI_1901_2014_all_TWS_NCA = get_regional_TWS_IPSL(33)

IWW_IRR_1901_2014_all_TWS_NCA_mean_IPSL = np.squeeze(IWW_IRR_1901_2014_all_TWS_NCA)


IWW_NOI_1901_2014_all_TWS_NCA_mean_IPSL = np.squeeze(IWW_NOI_1901_2014_all_TWS_NCA)


IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean_IPSL = np.squeeze(IWW_IRR_NOI_1901_2014_all_TWS_NCA)


In [ ]:
def plot_line_no_grd(ax, index, data_mean, data_min, data_max, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.4, linestyle='--')
#     ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)
    plt.legend(loc = 'best', fontsize = 16)
    
def plot_line_IPSL(ax, index, data_mean, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.4, linestyle=':')
#     ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)
    plt.legend(loc = 'best', fontsize = 16)
    
def plot_line_E3SM(ax, index, data_mean, title, title1, label, color, ylabel, xlabel, marker):
    ax.text(-0.05, 1.05, index, color='dimgrey', fontsize=16, transform=ax.transAxes, weight='bold')
    ax.plot(range(1901, 2015), data_mean, label = label, color = color, linewidth = 2, marker = marker, markersize=1.2, alpha=0.4, linestyle='-.')
#     ax.fill_between(range(1901, 2015), data_min, data_max, color = color, alpha=0.2)
    plt.title(title, loc='right', fontsize = 18)
    plt.title(title1, loc='left', fontsize = 18)
    plt.ylabel(ylabel, fontsize = 16)
    plt.xlabel(xlabel, fontsize = 16)
    plt.yticks(fontsize = 14)
    plt.xticks(fontsize=14)
    plt.xlim(1901,2014)
    plt.legend(loc = 'best', fontsize = 16)


fig = plt.figure(figsize=(12, 20),dpi=300)
fig.subplots_adjust(hspace=0.2, wspace=0.3, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)





ax1 = plt.subplot(421, frameon=True)
index = 'a'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_mean),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_mean)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_25)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_75)/365/86400,19,2), r'South Asia', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_mean)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_25)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_75)/365/86400,19,2), r'South Asia', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_75_no_grd)/365/86400,19,2), r'South Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_25_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_75_no_grd)/365/86400,19,2), r'South Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_mean_IPSL)/365/86400,19,2), r'South Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_mean_IPSL)/365/86400,19,2), r'South Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_SAS_mean_E3SM)/365/86400,19,2), r'South Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_SAS_mean_E3SM)/365/86400,19,2), r'South Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-1000, 1000])

ax1 = plt.subplot(422, frameon=True)
index = 'b'
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_25)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_75)/365/86400,19,2), r'South Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_75_no_grd)/365/86400,19,2), r'South Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean_IPSL)/365/86400,19,2), r'South Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_SAS_mean_E3SM)/365/86400,19,2), r'South Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-1000, 1000])




ax1 = plt.subplot(423, frameon=True)
index = 'c'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_mean),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_mean),19,2)/365/86400, savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_25)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_75)/365/86400,19,2), r'Mediterranean', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_mean),19,2)/365/86400, savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_25)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_75)/365/86400,19,2), r'Mediterranean', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_75_no_grd)/365/86400,19,2), r'Mediterranean', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_25_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_75_no_grd)/365/86400,19,2), r'Mediterranean', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_mean_IPSL)/365/86400,19,2), r'Mediterranean', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_mean_IPSL)/365/86400,19,2), r'Mediterranean', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_MED_mean_E3SM)/365/86400,19,2), r'Mediterranean', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_MED_mean_E3SM)/365/86400,19,2), r'Mediterranean', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-300, 300])

ax1 = plt.subplot(424, frameon=True)
index = 'd'
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_mean)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_25)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_75)/365/86400,19,2), r'Mediterranean', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_75_no_grd)/365/86400,19,2), r'Mediterranean', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_mean_IPSL)/365/86400,19,2), r'Mediterranean', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_MED_mean_E3SM)/365/86400,19,2), r'Mediterranean', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-300, 300])





ax1 = plt.subplot(425, frameon=True)
index = 'e'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_mean),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_mean)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_25)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_75)/365/86400,19,2), r'Central North America', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_mean)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_25)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_75)/365/86400,19,2), r'Central North America', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_75_no_grd)/365/86400,19,2), r'Central North America', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_25_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_75_no_grd)/365/86400,19,2), r'Central North America', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_mean_IPSL)/365/86400,19,2), r'Central North America', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_mean_IPSL)/365/86400,19,2), r'Central North America', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_CNA_mean_E3SM)/365/86400,19,2), r'Central North America', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_CNA_mean_E3SM)/365/86400,19,2), r'Central North America', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-1000, 1000])

ax1 = plt.subplot(426, frameon=True)
index = 'f'
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_25)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_75)/365/86400,19,2), r'Central North America', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_75_no_grd)/365/86400,19,2), r'Central North America', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean_IPSL)/365/86400,19,2), r'Central North America', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_CNA_mean_E3SM)/365/86400,19,2), r'Central North America', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-1000, 1000])



ax1 = plt.subplot(427, frameon=True)
index = 'g'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_mean),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_25),19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_75),19,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_mean)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_25)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_75)/365/86400,19,2), r'West Central Asia', '', 'tranirr', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_mean)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_25)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_75)/365/86400,19,2), r'West Central Asia', '', '1901irr', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_75_no_grd)/365/86400,19,2), r'West Central Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_25_no_grd)/365/86400,19,2), savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_75_no_grd)/365/86400,19,2), r'West Central Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_mean_IPSL)/365/86400,19,2), r'West Central Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_mean_IPSL)/365/86400,19,2), r'West Central Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_TWS_NCA_mean_E3SM)/365/86400,19,2), r'West Central Asia', '', '', 'dodgerblue', r'land TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_TWS_NCA_mean_E3SM)/365/86400,19,2), r'West Central Asia', '', '', 'brown', r'land TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-500, 500])

ax1 = plt.subplot(428, frameon=True)
index = 'h'
plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_25)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_75)/365/86400,19,2), r'West Central Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_no_grd(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_25_no_grd)/365/86400,19,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_75_no_grd)/365/86400,19,2), r'West Central Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_IPSL(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean_IPSL)/365/86400,19,2), r'West Central Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
plot_line_E3SM(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_TWS_NCA_mean_E3SM)/365/86400,19,2), r'West Central Asia', '', '', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([-500, 500])



In [10]:
def calcu_global_IWW(str, AREA, var):
    AREA = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/input_data/AREA.mat', 'AREA') # Area for CLM_data
    area_for_calcu = AREA.T
    
    
    IWW_1901_2014 = get_data_from_nc(str, var)
    area_for_calcu[np.isnan(IWW_1901_2014[0, :, :])] = np.nan
    IWW_globe_1901_2014 = IWW_1901_2014 * (AREA.T)
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    
    area_for_calcu = np.nansum(area_for_calcu, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=0)
    return IWW_globe_1901_2014/area_for_calcu

def calcu_global_IWW_region(str, AREA,region_id, var):
    AREA = get_data_from_mat('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/input_data/AREA.mat', 'AREA') # Area for CLM_data
    area_for_calcu = AREA.T
    
    
    area_for_calcu[ar6_region <= region_id-0.2] = np.nan
    area_for_calcu[ar6_region >= region_id+0.2] = np.nan
    
    IWW_1901_2014 = get_data_from_nc(str, var)
    IWW_globe_1901_2014 = IWW_1901_2014 * area_for_calcu
    area_for_calcu[np.isnan(IWW_1901_2014[0, :, :])] = np.nan
    
    IWW_globe_1901_2014[:, ar6_region<=region_id-0.2] = np.nan
    IWW_globe_1901_2014[:, ar6_region>=region_id+0.2] = np.nan
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    IWW_globe_1901_2014 = np.nansum(IWW_globe_1901_2014, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=1)
    area_for_calcu = np.nansum(area_for_calcu, axis=0)
    
    return IWW_globe_1901_2014/area_for_calcu

In [11]:
str_IWW_CESM2 = 'CESM2_IRR_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2 = calcu_global_IWW(str_IWW_CESM2, AREA, 'QIRRIG')
str_IWW_CESM2_NOI = 'CESM2_NOI_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2 = calcu_global_IWW(str_IWW_CESM2_NOI, AREA, 'QIRRIG')

str_IWW_CESM2_IRR01 = 'CESM2_IRR01_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2_IRR01 = calcu_global_IWW(str_IWW_CESM2_IRR01, AREA, 'QIRRIG')
str_IWW_CESM2_NOI01 = 'CESM2_NOI01_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2_IRR01 = calcu_global_IWW(str_IWW_CESM2_NOI01, AREA, 'QIRRIG')

str_IWW_CESM2_IRR02 = 'CESM2_IRR02_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2_IRR02 = calcu_global_IWW(str_IWW_CESM2_IRR02, AREA, 'QIRRIG')
str_IWW_CESM2_NOI02 = 'CESM2_NOI02_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2_IRR02 = calcu_global_IWW(str_IWW_CESM2_NOI02, AREA, 'QIRRIG')

str_IWW_CESM2_IRR03 = 'CESM2_IRR03_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2_IRR03 = calcu_global_IWW(str_IWW_CESM2_IRR03, AREA, 'QIRRIG')
str_IWW_CESM2_NOI03 = 'CESM2_NOI03_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2_IRR03 = calcu_global_IWW(str_IWW_CESM2_NOI03, AREA, 'QIRRIG')

IWW_IRR_1901_2014_CESM2 = (IWW_IRR_1901_2014_CESM2_IRR01 + IWW_IRR_1901_2014_CESM2_IRR02 + IWW_IRR_1901_2014_CESM2_IRR03)/3
IWW_NOI_1901_2014_CESM2 = (IWW_NOI_1901_2014_CESM2_IRR01 + IWW_NOI_1901_2014_CESM2_IRR02 + IWW_NOI_1901_2014_CESM2_IRR03)/3


str_IWW_CESM2_gw = 'CESM2_gw_IRR_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2_gw = calcu_global_IWW(str_IWW_CESM2_gw, AREA, 'QIRRIG')
str_IWW_CESM2_gw_NOI = 'CESM2_gw_NOI_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2_gw = calcu_global_IWW(str_IWW_CESM2_gw_NOI, AREA, 'QIRRIG')

str_IWW_CESM2_gw_IRR01 = 'CESM2_gw_IRR01_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2_gw_IRR01 = calcu_global_IWW(str_IWW_CESM2_gw_IRR01, AREA, 'QIRRIG')
str_IWW_CESM2_gw_NOI01 = 'CESM2_gw_NOI01_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2_gw_IRR01 = calcu_global_IWW(str_IWW_CESM2_gw_NOI01, AREA, 'QIRRIG')

str_IWW_CESM2_gw_IRR02 = 'CESM2_gw_IRR02_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2_gw_IRR02 = calcu_global_IWW(str_IWW_CESM2_gw_IRR02, AREA, 'QIRRIG')
str_IWW_CESM2_gw_NOI02 = 'CESM2_gw_NOI02_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2_gw_IRR02 = calcu_global_IWW(str_IWW_CESM2_gw_NOI02, AREA, 'QIRRIG')

str_IWW_CESM2_gw_IRR03 = 'CESM2_gw_IRR03_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_IRR_1901_2014_CESM2_gw_IRR03 = calcu_global_IWW(str_IWW_CESM2_gw_IRR03, AREA, 'QIRRIG')
str_IWW_CESM2_gw_NOI03 = 'CESM2_gw_NOI03_YEARLYQIRRIG_timmean_1901_2014.nc'
IWW_NOI_1901_2014_CESM2_gw_IRR03 = calcu_global_IWW(str_IWW_CESM2_gw_NOI03, AREA, 'QIRRIG')

IWW_IRR_1901_2014_CESM2_gw = (IWW_IRR_1901_2014_CESM2_gw_IRR01 + IWW_IRR_1901_2014_CESM2_gw_IRR02 + IWW_IRR_1901_2014_CESM2_gw_IRR03)/3
IWW_NOI_1901_2014_CESM2_gw = (IWW_NOI_1901_2014_CESM2_gw_IRR01 + IWW_NOI_1901_2014_CESM2_gw_IRR02 + IWW_NOI_1901_2014_CESM2_gw_IRR03)/3

str_IWW_NorESM = 'NorESM/QIRRIG/NorESM_IRR_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_IRR_1901_2014_NorESM = calcu_global_IWW(str_IWW_NorESM, AREA, 'QIRRIG')
str_IWW_NorESM_NOI = 'NorESM/QIRRIG/NorESM_NOI_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_NOI_1901_2014_NorESM = calcu_global_IWW(str_IWW_NorESM_NOI, AREA, 'QIRRIG')

str_IWW_NorESM_IRR01 = 'NorESM_IRR01_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_IRR_1901_2014_NorESM_IRR01 = calcu_global_IWW(str_IWW_NorESM_IRR01, AREA, 'QIRRIG')
str_IWW_NorESM_NOI01 = 'NorESM_NOI01_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_NOI_1901_2014_NorESM_IRR01 = calcu_global_IWW(str_IWW_NorESM_NOI01, AREA, 'QIRRIG')

str_IWW_NorESM_IRR02 = 'NorESM_IRR02_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_IRR_1901_2014_NorESM_IRR02 = calcu_global_IWW(str_IWW_NorESM_IRR02, AREA, 'QIRRIG')
str_IWW_NorESM_NOI02 = 'NorESM_NOI02_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_NOI_1901_2014_NorESM_IRR02 = calcu_global_IWW(str_IWW_NorESM_NOI02, AREA, 'QIRRIG')

str_IWW_NorESM_IRR03 = 'NorESM_IRR03_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_IRR_1901_2014_NorESM_IRR03 = calcu_global_IWW(str_IWW_NorESM_IRR03, AREA, 'QIRRIG')
str_IWW_NorESM_NOI03 = 'NorESM_NOI03_YEARLYQIRRIG_yearmean_1901_2014.nc'
IWW_NOI_1901_2014_NorESM_IRR03 = calcu_global_IWW(str_IWW_NorESM_NOI03, AREA, 'QIRRIG')

IWW_IRR_1901_2014_NorESM = (IWW_IRR_1901_2014_NorESM_IRR01 + IWW_IRR_1901_2014_NorESM_IRR02 + IWW_IRR_1901_2014_NorESM_IRR03)/3
IWW_NOI_1901_2014_NorESM = (IWW_IRR_1901_2014_NorESM_IRR01 + IWW_IRR_1901_2014_NorESM_IRR02 + IWW_IRR_1901_2014_NorESM_IRR03)/3

str_IWW_IPSL = 'IRR01_1901_2014_irr_Day_Month.nc_YEARLY_0.9x1.25'
IWW_IRR_1901_2014_IPSL = calcu_global_IWW(str_IWW_IPSL, AREA, 'irr')

str_IWW_IPSL_IRR01 = 'IRR01_1901_2014_irr_Day_Month.nc_YEARLY_0.9x1.25'
IWW_IRR_1901_2014_IPSL_IRR01 = calcu_global_IWW(str_IWW_IPSL_IRR01, AREA, 'irr')






str_IWW_CNRM = 'arrww_mon_CNRM-CM6-1_hist-irr_r1i1p1f2_gr_190101-201412.nc_yearmean_YEARLY_0.9x1.25'
IWW_IRR_1901_2014_CNRM = calcu_global_IWW(str_IWW_CNRM, AREA, 'airrww')
str_IWW_CNRM_NOI = 'airrww_mon_CNRM-CM6-1_hist-noirr_r1i1p1f2_gr_190101-201412.nc_yearmean_YEARLY_0.9x1.25'
IWW_NOI_1901_2014_CNRM = calcu_global_IWW(str_IWW_CNRM_NOI, AREA, 'airrww')

str_IWW_CNRM_IRR01 = 'CNRM-CM6-1_hist-irr_r1i1p1f2/after_processing/airrww_mon_CNRM-CM6-1_hist-irr_r1i1p1f2_gr_190101-201412.nc_yearmean_YEARLY_0.9x1.25'
IWW_IRR_1901_2014_CNRM_IRR01 = calcu_global_IWW(str_IWW_CNRM_IRR01, AREA, 'airrww')
str_IWW_CNRM_NOI01 = 'CNRM-CM6-1_hist-noirr_r1i1p1f2/after_processing/airrww_mon_CNRM-CM6-1_hist-noirr_r1i1p1f2_gr_190101-201412.nc_yearmean_YEARLY_0.9x1.25'
IWW_NOI_1901_2014_CNRM_NOI01 = calcu_global_IWW(str_IWW_CNRM_NOI01, AREA, 'airrww')

str_IWW_E3SM = 'IRR_QIRRIG_REAL_yearmean_1901_2014.nc_0.9x1.25_YEARLY_yearmean'
IWW_IRR_1901_2014_E3SM = calcu_global_IWW(str_IWW_E3SM, AREA, 'QIRRIG_REAL')
str_IWW_E3SM_NOI = 'NOI_QIRRIG_REAL_yearmean_1901_2014.nc_0.9x1.25_YEARLY_yearmean'
IWW_NOI_1901_2014_E3SM = calcu_global_IWW(str_IWW_E3SM_NOI, AREA, 'QIRRIG_REAL')

str_IWW_E3SM_IRR01 = 'IRR01_QIRRIG_REAL_yearmean_1901_2014.nc_0.9x1.25_YEARLY_yearmean'
IWW_IRR_1901_2014_E3SM_IRR01 = calcu_global_IWW(str_IWW_E3SM_IRR01, AREA, 'QIRRIG_REAL')
str_IWW_E3SM_IRR02 = 'IRR02_QIRRIG_REAL_yearmean_1901_2014.nc_0.9x1.25_YEARLY_yearmean'
IWW_IRR_1901_2014_E3SM_IRR02 = calcu_global_IWW(str_IWW_E3SM_IRR02, AREA, 'QIRRIG_REAL')
str_IWW_E3SM_NOI01 = 'NOI01_QIRRIG_REAL_yearmean_1901_2014.nc_0.9x1.25_YEARLY_yearmean'
IWW_IRR_1901_2014_E3SM_NOI01 = calcu_global_IWW(str_IWW_E3SM_NOI01, AREA, 'QIRRIG_REAL')
str_IWW_E3SM_NOI02 = 'NOI02_QIRRIG_REAL_yearmean_1901_2014.nc_0.9x1.25_YEARLY_yearmean'
IWW_IRR_1901_2014_E3SM_NOI02 = calcu_global_IWW(str_IWW_E3SM_NOI02, AREA, 'QIRRIG_REAL')

IWW_IRR_1901_2014_E3SM = (IWW_IRR_1901_2014_E3SM_IRR01 + IWW_IRR_1901_2014_E3SM_IRR02)/2
IWW_NOI_1901_2014_E3SM = (IWW_IRR_1901_2014_E3SM_NOI01 + IWW_IRR_1901_2014_E3SM_NOI02)/2

# str_IWW_MIROC = '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/irrac_mon_MIROC_IRR01_1901-2014.nc_0.9x1.25_yearmean'
# IWW_IRR_1901_2014_MIROC = calcu_global_IWW(str_IWW_MIROC, AREA, 'QIRRIG_REAL')
# str_IWW_MIROC_NOI = '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/irrac_mon_MIROC_NOI01_1901-2014.nc_0.9x1.25_yearmean'
# IWW_NOI_1901_2014_MIROC = calcu_global_IWW(str_IWW_MIROC_NOI, AREA, 'QIRRIG_REAL')


str_IWW_MIROC_IRR01 = 'irrac_mon_MIROC_IRR01_1901-2014.nc_0.9x1.25_yearmean'
IWW_IRR_1901_2014_MIROC_IRR01 = calcu_global_IWW(str_IWW_MIROC_IRR01, AREA, 'irrac')
str_IWW_MIROC_IRR02 = 'irrac_mon_MIROC_IRR02_1901-2014.nc_0.9x1.25_yearmean'
IWW_IRR_1901_2014_MIROC_IRR02 = calcu_global_IWW(str_IWW_MIROC_IRR02, AREA, 'irrac')
str_IWW_MIROC_IRR03 = 'irrac_mon_MIROC_IRR03_1901-2014.nc_0.9x1.25_yearmean'
IWW_IRR_1901_2014_MIROC_IRR03 = calcu_global_IWW(str_IWW_MIROC_IRR03, AREA, 'irrac')

str_IWW_MIROC_NOI01 = 'irrac_mon_MIROC_NOI01_1901-2014.nc_0.9x1.25_yearmean'
IWW_NOI_1901_2014_MIROC_IRR01 = calcu_global_IWW(str_IWW_MIROC_NOI01, AREA, 'irrac')
str_IWW_MIROC_NOI02 = 'irrac_mon_MIROC_NOI02_1901-2014.nc_0.9x1.25_yearmean'
IWW_NOI_1901_2014_MIROC_IRR02 = calcu_global_IWW(str_IWW_MIROC_NOI02, AREA, 'irrac')
str_IWW_MIROC_NOI03 = 'irrac_mon_MIROC_NOI03_1901-2014.nc_0.9x1.25_yearmean'
IWW_NOI_1901_2014_MIROC_IRR03 = calcu_global_IWW(str_IWW_MIROC_NOI03, AREA, 'irrac')

IWW_IRR_1901_2014_MIROC = (IWW_IRR_1901_2014_MIROC_IRR01 + IWW_IRR_1901_2014_MIROC_IRR02 + IWW_IRR_1901_2014_MIROC_IRR03) / 3
IWW_NOI_1901_2014_MIROC = (IWW_NOI_1901_2014_MIROC_IRR01 + IWW_NOI_1901_2014_MIROC_IRR02 + IWW_NOI_1901_2014_MIROC_IRR03) / 3

In [12]:
str_IWW_MIROC = '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/irrac_mon_MIROC_IRR01_1901-2014.nc_0.9x1.25_yearmean'
IWW_IRR_1901_2014_MIROC = calcu_global_IWW(str_IWW_MIROC, AREA, 'irrac')
str_IWW_MIROC_NOI = '/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/MIROC/irrac_mon_MIROC_NOI01_1901-2014.nc_0.9x1.25_yearmean'
IWW_NOI_1901_2014_MIROC = calcu_global_IWW(str_IWW_MIROC_NOI, AREA, 'irrac')

IWW_IRR_1901_2014_MIROC_WNA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 4, 'irrac')
IWW_IRR_1901_2014_MIROC_CNA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 5, 'irrac')
IWW_IRR_1901_2014_MIROC_NCA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 7, 'irrac')
IWW_IRR_1901_2014_MIROC_MED = calcu_global_IWW_region(str_IWW_MIROC, AREA, 20, 'irrac')
IWW_IRR_1901_2014_MIROC_WCA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 33, 'irrac')
IWW_IRR_1901_2014_MIROC_EAS = calcu_global_IWW_region(str_IWW_MIROC, AREA, 36, 'irrac')
IWW_IRR_1901_2014_MIROC_SAS = calcu_global_IWW_region(str_IWW_MIROC, AREA, 38, 'irrac')
IWW_IRR_1901_2014_MIROC_SEA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 39, 'irrac')

IWW_NOI_1901_2014_MIROC_WNA = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 4, 'irrac')
IWW_NOI_1901_2014_MIROC_CNA = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 5, 'irrac')
IWW_NOI_1901_2014_MIROC_NCA = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 7, 'irrac')
IWW_NOI_1901_2014_MIROC_MED = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 20, 'irrac')
IWW_NOI_1901_2014_MIROC_WCA = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 33, 'irrac')
IWW_NOI_1901_2014_MIROC_EAS = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 36, 'irrac')
IWW_NOI_1901_2014_MIROC_SAS = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 38, 'irrac')
IWW_NOI_1901_2014_MIROC_SEA = calcu_global_IWW_region(str_IWW_MIROC_NOI, AREA, 39, 'irrac')

In [13]:
IWW_IRR_1901_2014_CESM2_WNA = calcu_global_IWW_region(str_IWW_CESM2, AREA, 4, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_CNA = calcu_global_IWW_region(str_IWW_CESM2, AREA, 5, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_NCA = calcu_global_IWW_region(str_IWW_CESM2, AREA, 7, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_MED = calcu_global_IWW_region(str_IWW_CESM2, AREA, 20, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_WCA = calcu_global_IWW_region(str_IWW_CESM2, AREA, 33, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_EAS = calcu_global_IWW_region(str_IWW_CESM2, AREA, 36, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_SAS = calcu_global_IWW_region(str_IWW_CESM2, AREA, 38, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_SEA = calcu_global_IWW_region(str_IWW_CESM2, AREA, 39, 'QIRRIG')

IWW_IRR_1901_2014_CESM2_gw_WNA = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 4, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_gw_CNA = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 5, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_gw_NCA = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 7, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_gw_MED = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 20, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_gw_WCA = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 33, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_gw_EAS = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 36, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_gw_SAS = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 38, 'QIRRIG')
IWW_IRR_1901_2014_CESM2_gw_SEA = calcu_global_IWW_region(str_IWW_CESM2_gw, AREA, 39, 'QIRRIG')

IWW_IRR_1901_2014_NorESM_WNA = calcu_global_IWW_region(str_IWW_NorESM, AREA, 4, 'QIRRIG')
IWW_IRR_1901_2014_NorESM_CNA = calcu_global_IWW_region(str_IWW_NorESM, AREA, 5, 'QIRRIG')
IWW_IRR_1901_2014_NorESM_NCA = calcu_global_IWW_region(str_IWW_NorESM, AREA, 7, 'QIRRIG')
IWW_IRR_1901_2014_NorESM_MED = calcu_global_IWW_region(str_IWW_NorESM, AREA, 20, 'QIRRIG')
IWW_IRR_1901_2014_NorESM_WCA = calcu_global_IWW_region(str_IWW_NorESM, AREA, 33, 'QIRRIG')
IWW_IRR_1901_2014_NorESM_EAS = calcu_global_IWW_region(str_IWW_NorESM, AREA, 36, 'QIRRIG')
IWW_IRR_1901_2014_NorESM_SAS = calcu_global_IWW_region(str_IWW_NorESM, AREA, 38, 'QIRRIG')
IWW_IRR_1901_2014_NorESM_SEA = calcu_global_IWW_region(str_IWW_NorESM, AREA, 39, 'QIRRIG')

IWW_IRR_1901_2014_E3SM_WNA = calcu_global_IWW_region(str_IWW_E3SM, AREA, 4, 'QIRRIG_REAL')
IWW_IRR_1901_2014_E3SM_CNA = calcu_global_IWW_region(str_IWW_E3SM, AREA, 5, 'QIRRIG_REAL')
IWW_IRR_1901_2014_E3SM_NCA = calcu_global_IWW_region(str_IWW_E3SM, AREA, 7, 'QIRRIG_REAL')
IWW_IRR_1901_2014_E3SM_MED = calcu_global_IWW_region(str_IWW_E3SM, AREA, 20, 'QIRRIG_REAL')
IWW_IRR_1901_2014_E3SM_WCA = calcu_global_IWW_region(str_IWW_E3SM, AREA, 33, 'QIRRIG_REAL')
IWW_IRR_1901_2014_E3SM_EAS = calcu_global_IWW_region(str_IWW_E3SM, AREA, 36, 'QIRRIG_REAL')
IWW_IRR_1901_2014_E3SM_SAS = calcu_global_IWW_region(str_IWW_E3SM, AREA, 38, 'QIRRIG_REAL')
IWW_IRR_1901_2014_E3SM_SEA = calcu_global_IWW_region(str_IWW_E3SM, AREA, 39, 'QIRRIG_REAL')

IWW_IRR_1901_2014_CNRM_WNA = calcu_global_IWW_region(str_IWW_CNRM, AREA, 4, 'airrww')
IWW_IRR_1901_2014_CNRM_CNA = calcu_global_IWW_region(str_IWW_CNRM, AREA, 5, 'airrww')
IWW_IRR_1901_2014_CNRM_NCA = calcu_global_IWW_region(str_IWW_CNRM, AREA, 7, 'airrww')
IWW_IRR_1901_2014_CNRM_MED = calcu_global_IWW_region(str_IWW_CNRM, AREA, 20, 'airrww')
IWW_IRR_1901_2014_CNRM_WCA = calcu_global_IWW_region(str_IWW_CNRM, AREA, 33, 'airrww')
IWW_IRR_1901_2014_CNRM_EAS = calcu_global_IWW_region(str_IWW_CNRM, AREA, 36, 'airrww')
IWW_IRR_1901_2014_CNRM_SAS = calcu_global_IWW_region(str_IWW_CNRM, AREA, 38, 'airrww')
IWW_IRR_1901_2014_CNRM_SEA = calcu_global_IWW_region(str_IWW_CNRM, AREA, 39, 'airrww')

IWW_IRR_1901_2014_IPSL_WNA = calcu_global_IWW_region(str_IWW_IPSL, AREA, 4, 'irr')
IWW_IRR_1901_2014_IPSL_CNA = calcu_global_IWW_region(str_IWW_IPSL, AREA, 5, 'irr')
IWW_IRR_1901_2014_IPSL_NCA = calcu_global_IWW_region(str_IWW_IPSL, AREA, 7, 'irr')
IWW_IRR_1901_2014_IPSL_MED = calcu_global_IWW_region(str_IWW_IPSL, AREA, 20, 'irr')
IWW_IRR_1901_2014_IPSL_WCA = calcu_global_IWW_region(str_IWW_IPSL, AREA, 33, 'irr')
IWW_IRR_1901_2014_IPSL_EAS = calcu_global_IWW_region(str_IWW_IPSL, AREA, 36, 'irr')
IWW_IRR_1901_2014_IPSL_SAS = calcu_global_IWW_region(str_IWW_IPSL, AREA, 38, 'irr')
IWW_IRR_1901_2014_IPSL_SEA = calcu_global_IWW_region(str_IWW_IPSL, AREA, 39, 'irr')

# IWW_IRR_1901_2014_MIROC_WNA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 4, 'irrac')
# IWW_IRR_1901_2014_MIROC_CNA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 5, 'irrac')
# IWW_IRR_1901_2014_MIROC_NCA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 7, 'irrac')
# IWW_IRR_1901_2014_MIROC_MED = calcu_global_IWW_region(str_IWW_MIROC, AREA, 20, 'irrac')
# IWW_IRR_1901_2014_MIROC_WCA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 33, 'irrac')
# IWW_IRR_1901_2014_MIROC_EAS = calcu_global_IWW_region(str_IWW_MIROC, AREA, 36, 'irrac')
# IWW_IRR_1901_2014_MIROC_SAS = calcu_global_IWW_region(str_IWW_MIROC, AREA, 38, 'irrac')
# IWW_IRR_1901_2014_MIROC_SEA = calcu_global_IWW_region(str_IWW_MIROC, AREA, 39, 'irrac')

In [14]:
IWW_NOI_1901_2014_CESM2_WNA = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 4, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_CNA = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 5, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_NCA = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 7, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_MED = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 20, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_WCA = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 33, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_EAS = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 36, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_SAS = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 38, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_SEA = calcu_global_IWW_region(str_IWW_CESM2_NOI, AREA, 39, 'QIRRIG')

IWW_NOI_1901_2014_CESM2_gw_WNA = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 4, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_gw_CNA = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 5, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_gw_NCA = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 7, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_gw_MED = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 20, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_gw_WCA = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 33, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_gw_EAS = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 36, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_gw_SAS = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 38, 'QIRRIG')
IWW_NOI_1901_2014_CESM2_gw_SEA = calcu_global_IWW_region(str_IWW_CESM2_gw_NOI, AREA, 39, 'QIRRIG')

IWW_NOI_1901_2014_NorESM_WNA = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 4, 'QIRRIG')
IWW_NOI_1901_2014_NorESM_CNA = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 5, 'QIRRIG')
IWW_NOI_1901_2014_NorESM_NCA = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 7, 'QIRRIG')
IWW_NOI_1901_2014_NorESM_MED = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 20, 'QIRRIG')
IWW_NOI_1901_2014_NorESM_WCA = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 33, 'QIRRIG')
IWW_NOI_1901_2014_NorESM_EAS = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 36, 'QIRRIG')
IWW_NOI_1901_2014_NorESM_SAS = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 38, 'QIRRIG')
IWW_NOI_1901_2014_NorESM_SEA = calcu_global_IWW_region(str_IWW_NorESM_NOI, AREA, 39, 'QIRRIG')

IWW_NOI_1901_2014_E3SM_WNA = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 4, 'QIRRIG_REAL')
IWW_NOI_1901_2014_E3SM_CNA = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 5, 'QIRRIG_REAL')
IWW_NOI_1901_2014_E3SM_NCA = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 7, 'QIRRIG_REAL')
IWW_NOI_1901_2014_E3SM_MED = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 20, 'QIRRIG_REAL')
IWW_NOI_1901_2014_E3SM_WCA = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 33, 'QIRRIG_REAL')
IWW_NOI_1901_2014_E3SM_EAS = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 36, 'QIRRIG_REAL')
IWW_NOI_1901_2014_E3SM_SAS = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 38, 'QIRRIG_REAL')
IWW_NOI_1901_2014_E3SM_SEA = calcu_global_IWW_region(str_IWW_E3SM_NOI, AREA, 39, 'QIRRIG_REAL')

IWW_NOI_1901_2014_CNRM_WNA = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 4, 'airrww')
IWW_NOI_1901_2014_CNRM_CNA = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 5, 'airrww')
IWW_NOI_1901_2014_CNRM_NCA = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 7, 'airrww')
IWW_NOI_1901_2014_CNRM_MED = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 20, 'airrww')
IWW_NOI_1901_2014_CNRM_WCA = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 33, 'airrww')
IWW_NOI_1901_2014_CNRM_EAS = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 36, 'airrww')
IWW_NOI_1901_2014_CNRM_SAS = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 38, 'airrww')
IWW_NOI_1901_2014_CNRM_SEA = calcu_global_IWW_region(str_IWW_CNRM_NOI, AREA, 39, 'airrww')

In [15]:
# IWW_IRR_1901_2014_all_IWW = np.vstack((IWW_IRR_1901_2014_CESM2_IWW_global,
#                            IWW_IRR_1901_2014_CESM2_gw_IWW_global,
#                            IWW_IRR_1901_2014_NorESM_IWW_global,
#                            IWW_IRR_1901_2014_E3SM_IWW_global,
#                            IWW_IRR_1901_2014_IPSL_IWW_global,
#                            IWW_IRR_1901_2014_CNRM_IWW_global))



# IWW_NOI_1901_2014_all_IWW = np.vstack((IWW_NOI_1901_2014_CESM2_IWW_global,
#                                IWW_NOI_1901_2014_CESM2_gw_IWW_global,
#                                IWW_NOI_1901_2014_NorESM_IWW_global,
#                                IWW_NOI_1901_2014_E3SM_IWW_global,
#                                IWW_NOI_1901_2014_IPSL_IWW_global,
#                                IWW_NOI_1901_2014_CNRM_IWW_global))

IWW_IRR_1901_2014_all_IWW = np.vstack((
                           IWW_IRR_1901_2014_CESM2_gw[:-1],
                           
                           
                           IWW_IRR_1901_2014_IPSL,
                           IWW_IRR_1901_2014_CNRM))


IWW_NOI_1901_2014_IPSL = IWW_IRR_1901_2014_IPSL - IWW_IRR_1901_2014_IPSL
IWW_NOI_1901_2014_all_IWW = np.vstack((
                               IWW_NOI_1901_2014_CESM2_gw[:-1],
                               
                               IWW_NOI_1901_2014_IPSL,
                               IWW_NOI_1901_2014_CNRM))

# IWW_IRR_1901_2014_all_IWW = np.vstack((IWW_IRR_1901_2014_CESM2_gw_IWW_global,
#                            IWW_IRR_1901_2014_E3SM_IWW_global,
#                            IWW_IRR_1901_2014_IPSL_IWW_global))



# IWW_NOI_1901_2014_all_IWW = np.vstack((IWW_NOI_1901_2014_CESM2_gw_IWW_global,
#                                IWW_NOI_1901_2014_E3SM_IWW_global,
#                                IWW_NOI_1901_2014_IPSL_IWW_global))



IWW_IRR_NOI_1901_2014_all_IWW = IWW_IRR_1901_2014_all_IWW - IWW_NOI_1901_2014_all_IWW

IWW_IRR_1901_2014_all_IWW_75 = np.percentile(IWW_IRR_1901_2014_all_IWW, 75, axis = 0)
IWW_IRR_1901_2014_all_IWW_50 = np.percentile(IWW_IRR_1901_2014_all_IWW, 50, axis = 0)
IWW_IRR_1901_2014_all_IWW_25 = np.percentile(IWW_IRR_1901_2014_all_IWW, 25, axis = 0)


IWW_NOI_1901_2014_all_IWW_75 = np.percentile(IWW_NOI_1901_2014_all_IWW, 75, axis = 0)
IWW_NOI_1901_2014_all_IWW_50 = np.percentile(IWW_NOI_1901_2014_all_IWW, 50, axis = 0)
IWW_NOI_1901_2014_all_IWW_25 = np.percentile(IWW_NOI_1901_2014_all_IWW, 25, axis = 0)

IWW_IRR_NOI_1901_2014_all_IWW_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_IWW, 75, axis = 0)
IWW_IRR_NOI_1901_2014_all_IWW_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_IWW, 50, axis = 0)
IWW_IRR_NOI_1901_2014_all_IWW_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_IWW, 25, axis = 0)


In [26]:
IWW_IRR_1901_2014_all_SAS = np.vstack((
                            IWW_IRR_1901_2014_CESM2_SAS[:-1],
                           IWW_IRR_1901_2014_CESM2_gw_SAS[:-1],
                            IWW_IRR_1901_2014_NorESM_SAS[:-1],
                           IWW_IRR_1901_2014_IPSL_SAS,
                            IWW_IRR_1901_2014_E3SM_SAS[:-1],
                            IWW_IRR_1901_2014_MIROC_SAS,
                           IWW_IRR_1901_2014_CNRM_SAS))


IWW_NOI_1901_2014_IPSL_SAS = IWW_IRR_1901_2014_IPSL_SAS - IWW_IRR_1901_2014_IPSL_SAS
IWW_NOI_1901_2014_all_SAS = np.vstack((
                               IWW_NOI_1901_2014_CESM2_SAS[:-1],
                           IWW_NOI_1901_2014_CESM2_gw_SAS[:-1],
                            IWW_NOI_1901_2014_NorESM_SAS[:-1],
                           IWW_NOI_1901_2014_IPSL_SAS,
                            IWW_NOI_1901_2014_E3SM_SAS[:-1],
                            IWW_NOI_1901_2014_MIROC_SAS,
                           IWW_NOI_1901_2014_CNRM_SAS))


IWW_IRR_1901_2014_all_SAS_75 = np.percentile(IWW_IRR_1901_2014_all_SAS, 75, axis = 0)
IWW_IRR_1901_2014_all_SAS_50 = np.percentile(IWW_IRR_1901_2014_all_SAS, 50, axis = 0)
IWW_IRR_1901_2014_all_SAS_25 = np.percentile(IWW_IRR_1901_2014_all_SAS, 25, axis = 0)
IWW_IRR_1901_2014_all_SAS_mean = np.mean(IWW_IRR_1901_2014_all_SAS, axis = 0)

IWW_NOI_1901_2014_all_SAS_75 = np.percentile(IWW_NOI_1901_2014_all_SAS, 75, axis = 0)
IWW_NOI_1901_2014_all_SAS_50 = np.percentile(IWW_NOI_1901_2014_all_SAS, 50, axis = 0)
IWW_NOI_1901_2014_all_SAS_25 = np.percentile(IWW_NOI_1901_2014_all_SAS, 25, axis = 0)
IWW_NOI_1901_2014_all_SAS_mean = np.mean(IWW_NOI_1901_2014_all_SAS, axis = 0)

# IWW_IRR_NOI_1901_2014_all_SAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_SAS, 75, axis = 0)
# IWW_IRR_NOI_1901_2014_all_SAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_SAS, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_SAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_SAS, 25, axis = 0)

# IWW_IRR_1901_2014_all_EAS, IWW_NOI_1901_2014_all_EAS, IWW_IRR_NOI_1901_2014_all_EAS = get_regional(36)
# IWW_IRR_1901_2014_all_EAS_75 = np.percentile(IWW_IRR_1901_2014_all_EAS, 75, axis = 0)
# IWW_IRR_1901_2014_all_EAS_50 = np.percentile(IWW_IRR_1901_2014_all_EAS, 50, axis = 0)
# IWW_IRR_1901_2014_all_EAS_25 = np.percentile(IWW_IRR_1901_2014_all_EAS, 25, axis = 0)

# IWW_NOI_1901_2014_all_EAS_75 = np.percentile(IWW_NOI_1901_2014_all_EAS, 75, axis = 0)
# IWW_NOI_1901_2014_all_EAS_50 = np.percentile(IWW_NOI_1901_2014_all_EAS, 50, axis = 0)
# IWW_NOI_1901_2014_all_EAS_25 = np.percentile(IWW_NOI_1901_2014_all_EAS, 25, axis = 0)

# IWW_IRR_NOI_1901_2014_all_EAS_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_EAS, 75, axis = 0)
# IWW_IRR_NOI_1901_2014_all_EAS_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_EAS, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_EAS_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_EAS, 25, axis = 0)

IWW_IRR_1901_2014_all_MED = np.vstack((
                            IWW_IRR_1901_2014_CESM2_MED[:-1],
                           IWW_IRR_1901_2014_CESM2_gw_MED[:-1],
                            IWW_IRR_1901_2014_NorESM_MED[:-1],
                           IWW_IRR_1901_2014_IPSL_MED,
                            IWW_IRR_1901_2014_E3SM_MED[:-1],
                            IWW_IRR_1901_2014_MIROC_MED,
                           IWW_IRR_1901_2014_CNRM_MED))


IWW_NOI_1901_2014_IPSL_MED = IWW_IRR_1901_2014_IPSL_MED - IWW_IRR_1901_2014_IPSL_MED
IWW_NOI_1901_2014_all_MED = np.vstack((
                               IWW_NOI_1901_2014_CESM2_MED[:-1],
                           IWW_NOI_1901_2014_CESM2_gw_MED[:-1],
                            IWW_NOI_1901_2014_NorESM_MED[:-1],
                           IWW_NOI_1901_2014_IPSL_MED,
                            IWW_NOI_1901_2014_E3SM_MED[:-1],
                            IWW_NOI_1901_2014_MIROC_MED,
                           IWW_NOI_1901_2014_CNRM_MED))

IWW_IRR_1901_2014_all_MED_75 = np.percentile(IWW_IRR_1901_2014_all_MED, 75, axis = 0)
IWW_IRR_1901_2014_all_MED_50 = np.percentile(IWW_IRR_1901_2014_all_MED, 50, axis = 0)
IWW_IRR_1901_2014_all_MED_25 = np.percentile(IWW_IRR_1901_2014_all_MED, 25, axis = 0)
IWW_IRR_1901_2014_all_MED_mean = np.mean(IWW_IRR_1901_2014_all_MED, axis = 0)

IWW_NOI_1901_2014_all_MED_75 = np.percentile(IWW_NOI_1901_2014_all_MED, 75, axis = 0)
IWW_NOI_1901_2014_all_MED_50 = np.percentile(IWW_NOI_1901_2014_all_MED, 50, axis = 0)
IWW_NOI_1901_2014_all_MED_25 = np.percentile(IWW_NOI_1901_2014_all_MED, 25, axis = 0)
IWW_NOI_1901_2014_all_MED_mean = np.mean(IWW_NOI_1901_2014_all_MED, axis = 0)

# IWW_IRR_NOI_1901_2014_all_MED_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_MED, 75, axis = 0)
# IWW_IRR_NOI_1901_2014_all_MED_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_MED, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_MED_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_MED, 25, axis = 0)



# IWW_IRR_1901_2014_all_WNA, IWW_NOI_1901_2014_all_WNA, IWW_IRR_NOI_1901_2014_all_WNA = get_regional(4)
# IWW_IRR_1901_2014_all_WNA_75 = np.percentile(IWW_IRR_1901_2014_all_WNA, 75, axis = 0)
# IWW_IRR_1901_2014_all_WNA_50 = np.percentile(IWW_IRR_1901_2014_all_WNA, 50, axis = 0)
# IWW_IRR_1901_2014_all_WNA_25 = np.percentile(IWW_IRR_1901_2014_all_WNA, 25, axis = 0)

# IWW_NOI_1901_2014_all_WNA_75 = np.percentile(IWW_NOI_1901_2014_all_WNA, 75, axis = 0)
# IWW_NOI_1901_2014_all_WNA_50 = np.percentile(IWW_NOI_1901_2014_all_WNA, 50, axis = 0)
# IWW_NOI_1901_2014_all_WNA_25 = np.percentile(IWW_NOI_1901_2014_all_WNA, 25, axis = 0)

# IWW_IRR_NOI_1901_2014_all_WNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_WNA, 75, axis = 0)
# IWW_IRR_NOI_1901_2014_all_WNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_WNA, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_WNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_WNA, 25, axis = 0)


IWW_IRR_1901_2014_all_CNA = np.vstack((
                            IWW_IRR_1901_2014_CESM2_CNA[:-1],
                           IWW_IRR_1901_2014_CESM2_gw_CNA[:-1],
                            IWW_IRR_1901_2014_NorESM_CNA[:-1],
                           IWW_IRR_1901_2014_IPSL_CNA,
                            IWW_IRR_1901_2014_E3SM_CNA[:-1],
                            IWW_IRR_1901_2014_MIROC_CNA,
                           IWW_IRR_1901_2014_CNRM_CNA))


IWW_NOI_1901_2014_IPSL_CNA = IWW_IRR_1901_2014_IPSL_CNA - IWW_IRR_1901_2014_IPSL_CNA
IWW_NOI_1901_2014_all_CNA = np.vstack((
                               IWW_NOI_1901_2014_CESM2_CNA[:-1],
                           IWW_NOI_1901_2014_CESM2_gw_CNA[:-1],
                            IWW_NOI_1901_2014_NorESM_CNA[:-1],
                           IWW_NOI_1901_2014_IPSL_CNA,
                            IWW_NOI_1901_2014_E3SM_CNA[:-1],
                            IWW_NOI_1901_2014_MIROC_CNA,
                           IWW_NOI_1901_2014_CNRM_CNA))

IWW_IRR_1901_2014_all_CNA_75 = np.percentile(IWW_IRR_1901_2014_all_CNA, 75, axis = 0)
IWW_IRR_1901_2014_all_CNA_50 = np.percentile(IWW_IRR_1901_2014_all_CNA, 50, axis = 0)
IWW_IRR_1901_2014_all_CNA_25 = np.percentile(IWW_IRR_1901_2014_all_CNA, 25, axis = 0)
IWW_IRR_1901_2014_all_CNA_mean = np.mean(IWW_IRR_1901_2014_all_CNA, axis = 0)

IWW_NOI_1901_2014_all_CNA_75 = np.percentile(IWW_NOI_1901_2014_all_CNA, 75, axis = 0)
IWW_NOI_1901_2014_all_CNA_50 = np.percentile(IWW_NOI_1901_2014_all_CNA, 50, axis = 0)
IWW_NOI_1901_2014_all_CNA_25 = np.percentile(IWW_NOI_1901_2014_all_CNA, 25, axis = 0)
IWW_NOI_1901_2014_all_CNA_mean = np.mean(IWW_NOI_1901_2014_all_CNA, axis = 0)

# IWW_IRR_NOI_1901_2014_all_CNA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_CNA, 75, axis = 0)
# IWW_IRR_NOI_1901_2014_all_CNA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_CNA, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_CNA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_CNA, 25, axis = 0)


IWW_IRR_1901_2014_all_WCA = np.vstack((
                            IWW_IRR_1901_2014_CESM2_WCA[:-1],
                           IWW_IRR_1901_2014_CESM2_gw_WCA[:-1],
                            IWW_IRR_1901_2014_NorESM_WCA[:-1],
                           IWW_IRR_1901_2014_IPSL_WCA,
                            IWW_IRR_1901_2014_E3SM_WCA[:-1],
                            IWW_IRR_1901_2014_MIROC_WCA,
                           IWW_IRR_1901_2014_CNRM_WCA))


IWW_NOI_1901_2014_IPSL_WCA = IWW_IRR_1901_2014_IPSL_WCA - IWW_IRR_1901_2014_IPSL_WCA
IWW_NOI_1901_2014_all_WCA = np.vstack((
                               IWW_NOI_1901_2014_CESM2_WCA[:-1],
                           IWW_NOI_1901_2014_CESM2_gw_WCA[:-1],
                            IWW_NOI_1901_2014_NorESM_WCA[:-1],
                           IWW_NOI_1901_2014_IPSL_WCA,
                            IWW_NOI_1901_2014_E3SM_WCA[:-1],
                            IWW_NOI_1901_2014_MIROC_WCA,
                           IWW_NOI_1901_2014_CNRM_WCA))

IWW_IRR_1901_2014_all_WCA_75 = np.percentile(IWW_IRR_1901_2014_all_WCA, 75, axis = 0)
IWW_IRR_1901_2014_all_WCA_50 = np.percentile(IWW_IRR_1901_2014_all_WCA, 50, axis = 0)
IWW_IRR_1901_2014_all_WCA_25 = np.percentile(IWW_IRR_1901_2014_all_WCA, 25, axis = 0)
IWW_IRR_1901_2014_all_WCA_mean = np.mean(IWW_IRR_1901_2014_all_WCA, axis = 0)

IWW_NOI_1901_2014_all_WCA_75 = np.percentile(IWW_NOI_1901_2014_all_WCA, 75, axis = 0)
IWW_NOI_1901_2014_all_WCA_50 = np.percentile(IWW_NOI_1901_2014_all_WCA, 50, axis = 0)
IWW_NOI_1901_2014_all_WCA_25 = np.percentile(IWW_NOI_1901_2014_all_WCA, 25, axis = 0)
IWW_NOI_1901_2014_all_WCA_mean = np.mean(IWW_NOI_1901_2014_all_WCA, axis = 0)

# IWW_IRR_NOI_1901_2014_all_NCA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_NCA, 75, axis = 0)
# IWW_IRR_NOI_1901_2014_all_NCA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_NCA, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_NCA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_NCA, 25, axis = 0)


# IWW_IRR_1901_2014_all_SEA, IWW_NOI_1901_2014_all_SEA, IWW_IRR_NOI_1901_2014_all_SEA = get_regional(39)
# IWW_IRR_1901_2014_all_SEA_75 = np.percentile(IWW_IRR_1901_2014_all_SEA, 75, axis = 0)
# IWW_IRR_1901_2014_all_SEA_50 = np.percentile(IWW_IRR_1901_2014_all_SEA, 50, axis = 0)
# IWW_IRR_1901_2014_all_SEA_25 = np.percentile(IWW_IRR_1901_2014_all_SEA, 25, axis = 0)

# IWW_NOI_1901_2014_all_SEA_75 = np.percentile(IWW_NOI_1901_2014_all_SEA, 75, axis = 0)
# IWW_NOI_1901_2014_all_SEA_50 = np.percentile(IWW_NOI_1901_2014_all_SEA, 50, axis = 0)
# IWW_NOI_1901_2014_all_SEA_25 = np.percentile(IWW_NOI_1901_2014_all_SEA, 25, axis = 0)

# IWW_IRR_NOI_1901_2014_all_SEA_75 = np.percentile(IWW_IRR_NOI_1901_2014_all_SEA, 75, axis = 0)
# IWW_IRR_NOI_1901_2014_all_SEA_50 = np.percentile(IWW_IRR_NOI_1901_2014_all_SEA, 50, axis = 0)
# IWW_IRR_NOI_1901_2014_all_SEA_25 = np.percentile(IWW_IRR_NOI_1901_2014_all_SEA, 25, axis = 0)

In [ ]:
fig = plt.figure(figsize=(24, 4),dpi=300)
fig.subplots_adjust(hspace=0.2, wspace=0.3, left = 0.05, right = 0.95, top = 0.95, bottom = 0.05)





ax1 = plt.subplot(141, frameon=True)
index = 'a'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_75),15,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_SAS_mean),15,2), savgol_filter((IWW_IRR_1901_2014_all_SAS_25),15,2), savgol_filter((IWW_IRR_1901_2014_all_SAS_75),15,2), r'South Asia', '', 'tranirr', 'dodgerblue', r'IWW ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_SAS_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_SAS_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_SAS_75),15,2), r'South Asia', '', '1901irr', 'brown', r'IWW ($\mathregular{mm/year}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([0, 120])

# ax1 = plt.subplot(242, frameon=True)
# index = 'd'
# plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_SAS_mean),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_SAS_25),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_SAS_75),15,2), r'South Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plt.grid(linestyle = '--')
# plt.ylim([-1000, 1000])




ax1 = plt.subplot(142, frameon=True)
index = 'b'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_75),15,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_MED_mean),15,2), savgol_filter((IWW_IRR_1901_2014_all_MED_25),15,2), savgol_filter((IWW_IRR_1901_2014_all_MED_75),15,2), r'Mediterranean', '', 'tranirr', 'dodgerblue', r'IWW ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_MED_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_MED_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_MED_75),15,2), r'Mediterranean', '', '1901irr', 'brown', r'IWW ($\mathregular{mm/year}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([0, 40])

# ax1 = plt.subplot(244, frameon=True)
# index = 'f'
# plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_MED_mean),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_MED_25),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_MED_75),15,2), r'Mediterranean', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plt.grid(linestyle = '--')
# plt.ylim([-100, 100])





ax1 = plt.subplot(143, frameon=True)
index = 'c'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_75),15,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_CNA_mean),15,2), savgol_filter((IWW_IRR_1901_2014_all_CNA_25),15,2), savgol_filter((IWW_IRR_1901_2014_all_CNA_75),15,2), r'Central North America', '', 'tranirr', 'dodgerblue', r'IWW ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_CNA_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_CNA_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_CNA_75),15,2), r'Central North America', '', '1901irr', 'brown', r'IWW ($\mathregular{mm/year}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([0, 30])

# ax1 = plt.subplot(246, frameon=True)
# index = 'h'
# plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_CNA_mean),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_CNA_25),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_CNA_75),15,2), r'Central North America', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plt.grid(linestyle = '--')
# plt.ylim([-500, 500])



ax1 = plt.subplot(144, frameon=True)
index = 'd'
# plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_75),15,2), r'all land', 'P-ET', '1901irr', 'brown', 'hours', '', '')
plot_line(ax1, index, savgol_filter((IWW_IRR_1901_2014_all_WCA_mean),15,2), savgol_filter((IWW_IRR_1901_2014_all_WCA_25),15,2), savgol_filter((IWW_IRR_1901_2014_all_WCA_75),15,2), r'West Central Asia', '', 'tranirr', 'dodgerblue', r'IWW ($\mathregular{mm/year}$)', '', '')
plot_line(ax1, index, savgol_filter((IWW_NOI_1901_2014_all_WCA_mean),15,2), savgol_filter((IWW_NOI_1901_2014_all_WCA_25),15,2), savgol_filter((IWW_NOI_1901_2014_all_WCA_75),15,2), r'West Central Asia', '', '1901irr', 'brown', r'IWW ($\mathregular{mm/year}$)', '', '')

plt.grid(linestyle = '--')
plt.ylim([0, 60])

# ax1 = plt.subplot(248, frameon=True)
# index = 'j'
# plot_line(ax1, index, savgol_filter((IWW_IRR_NOI_1901_2014_all_NCA_mean),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_NCA_25),15,2), savgol_filter((IWW_IRR_NOI_1901_2014_all_NCA_75),15,2), r'West Central Asia', '', 'tranirr-1901irr', 'green', r'land $\Delta$TWS ($\mathregular{mm}$)', '', '')
# plt.grid(linestyle = '--')
# plt.ylim([-200, 200])






